In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import warnings

warnings.filterwarnings('ignore')

# --- (辅助函数 - 来自 Cell 1) ---
def parse_build_year(s):
    if pd.isnull(s): return np.nan
    s = str(s); years = re.findall(r'(\d{4})', s)
    if not years: return np.nan
    return np.mean([float(y) for y in years]) if len(years) > 1 else float(years[0])

def parse_first_number(s, default_val=np.nan):
    if pd.isnull(s): return default_val
    s = str(s).replace('暂无', '0')
    match = re.search(r'(\d+\.?\d*)', s)
    return float(match.group(1)) if match else default_val


In [2]:
# --- STAGE 1: 加载并进行通用预处理 (来自 Cell 2-5) ---
print("--- STAGE 1: 加载并进行通用预处理... ---")
try:
    df_combined = pd.read_csv('combined_price.csv') 
except FileNotFoundError:
    print("错误: 未找到 'combined_price.csv'。")
    # exit()

# 目标变量
if 'Price' in df_combined.columns:
    print("--- 正在转换 Price 为 log_price... ---")
    df_combined['log_price'] = np.log1p(df_combined['Price']) 
    df_combined = df_combined.drop('Price', axis=1, errors='ignore')

# --- (通用解析 - 不依赖统计) ---
# (这些解析逻辑对 train 和 test 同样适用，在拆分前完成)
# --- 供水供电重新编码 (新增代码) ---
if '供水' in df_combined.columns:
    # 将混合类别拆分为独立特征
    df_combined['供水_商水'] = df_combined['供水'].str.contains('商水', na=False).astype(int)
    df_combined['供水_民水'] = df_combined['供水'].str.contains('民水', na=False).astype(int)
    # 删除原始的供水列，避免后续One-Hot编码产生混合类别
    df_combined = df_combined.drop('供水', axis=1)

if '供电' in df_combined.columns:
    df_combined['供电_商电'] = df_combined['供电'].str.contains('商电', na=False).astype(int)
    df_combined['供电_民电'] = df_combined['供电'].str.contains('民电', na=False).astype(int)
    df_combined = df_combined.drop('供电', axis=1)

# 面积 (来自 Cell 4)
if '建筑面积' in df_combined.columns:
    df_combined['建筑面积'] = df_combined['建筑面积'].astype(str).str.replace('㎡', '', regex=False)
    df_combined['log_建筑面积'] = np.log1p(pd.to_numeric(df_combined['建筑面积'], errors='coerce'))

# 梯户比例 (来自 Cell 4)
if '梯户比例' in df_combined.columns:
    chinese_num_map = { "一": "1", "二": "2", "两": "2", "三": "3", "四": "4", "五": "5", "六": "6", "七": "7", "八": "8", "九": "9", "十": "10", "十一": "11", "十二": "12", "十三": "13", "十四": "14", "十五": "15", "十六": "16", "十七": "17", "十八": "18", "十九": "19", "二十": "20", "二十一": "21", "二十二": "22", "二十三": "23", "二十四": "24", "二十五": "25", "二十六": "26", "二十七": "27", "二十八": "28", "二十九": "29", "三十": "30", "三十一": "31", "三十二": "32", "三十三": "33", "三十四": "34", "三十五": "35", "三十六": "36", "三十七": "37", "三十八": "38", "三十九": "39", "四十": "40", "四十一": "41", "四十二": "42", "四十三": "43", "四十四": "44", "四十五": "45", "四十六": "46", "四十七": "47", "四十八": "48", "四十九": "49", "五十": "50", "五十一": "51", "五十二": "52", "五十三": "53", "五十四": "54", "五十五": "55", "五十六": "56", "五十七": "57", "五十八": "58", "五十九": "59", "六十": "60", "六十一": "61", "六十二": "62", "六十三": "63", "六十四": "64", "六十五": "65", "六十六": "66", "六十七": "67", "六十八": "68", "六十九": "69", "七十": "70" }
    temp_col = df_combined['梯户比例'].astype(str)
    sorted_keys = sorted(chinese_num_map.keys(), key=len, reverse=True)
    for char in sorted_keys: num_str = chinese_num_map[char]; temp_col = temp_col.str.replace(char, num_str)
    df_combined['梯'] = temp_col.str.extract(r'(\d+)梯').astype(float)
    df_combined['户'] = temp_col.str.extract(r'(\d+)户').astype(float)

# 所在楼层 (来自 Cell 4)
if '所在楼层' in df_combined.columns:
    floor_str = df_combined['所在楼层'].astype(str) 
    df_combined['总楼层_temp'] = floor_str.str.extract(r'共(\d+)层')[0].astype(float)
    floor_position_str = floor_str.str.extract(r'(地下室|底层|顶层|低楼层|中楼层|高楼层)')[0]
    floor_position_map = {'地下室': -1, '底层': 1, '低楼层': 2, '中楼层': 3, '高楼层': 4, '顶层': 5}
    df_combined['楼层位置_mapped'] = floor_position_str.map(floor_position_map)

# 房屋朝向 (来自 Cell 5)
if '房屋朝向' in df_combined.columns:
    df_combined['朝南'] = df_combined['房屋朝向'].str.contains('南', na=False).astype(int)
    df_combined['朝北'] = df_combined['房屋朝向'].str.contains('北', na=False).astype(int)
    df_combined['朝东'] = df_combined['房屋朝向'].str.contains('东', na=False).astype(int)
    df_combined['朝西'] = df_combined['房屋朝向'].str.contains('西', na=False).astype(int)

# 高缺失率数值 (仅解析) (来自 Cell 5)
if '建筑年代' in df_combined.columns: df_combined['建筑年代_parsed'] = df_combined['建筑年代'].apply(parse_build_year)
if '房屋总数' in df_combined.columns: df_combined['房屋总数_num'] = pd.to_numeric(df_combined['房屋总数'].astype(str).str.replace('户', '', regex=False), errors='coerce')
if '楼栋总数' in df_combined.columns: df_combined['楼栋总数_num'] = pd.to_numeric(df_combined['楼栋总数'].astype(str).str.replace('栋', '', regex=False), errors='coerce')
if '绿 化 率' in df_combined.columns: df_combined['绿化率_num'] = pd.to_numeric(df_combined['绿 化 率'].astype(str).str.replace('%', '', regex=False), errors='coerce') / 100.0
if '容 积 率' in df_combined.columns: df_combined['容积率_num'] = pd.to_numeric(df_combined['容 积 率'], errors='coerce')
if '燃气费' in df_combined.columns: df_combined['燃气费_num'] = df_combined['燃气费'].apply(parse_first_number)
if '停车位' in df_combined.columns: df_combined['停车位_num'] = pd.to_numeric(df_combined['停车位'], errors='coerce')
if '停车费用' in df_combined.columns: df_combined['停车费用_num'] = df_combined['停车费用'].apply(parse_first_number, default_val=0)

# 日期 (仅解析) (来自 Cell 5)
if '交易时间' in df_combined.columns: df_combined['交易时间'] = pd.to_datetime(df_combined['交易时间'], errors='coerce')
if '上次交易' in df_combined.columns: df_combined['上次交易'] = pd.to_datetime(df_combined['上次交易'], errors='coerce')

# 文本标志 (来自 Cell 6)
text_cols = ['房屋优势', '周边配套', '交通出行', '核心卖点', '客户反馈']
for col in text_cols:
    if col in df_combined.columns:
        df_combined[f'有_{col}'] = df_combined[col].notnull().astype(int)
        if col == '核心卖点':
             df_combined[f'{col}_长度'] = df_combined[col].str.len().fillna(0)

if '物业类别' in df_combined.columns: df_combined['物业类别'] = df_combined['物业类别'].astype(str)

# --- 特征工程: 解析 房屋户型 (提取 室, 厅, 卫) ---
if '房屋户型' in df_combined.columns:
    print("--- 正在解析 房屋户型 (提取 室, 厅, 卫)... ---")
    # 为了稳健地计算众数，先临时填充一下原始列的 NaN (可以用最高频的值)
    huxing_mode = df_combined['房屋户型'].mode()[0] if not df_combined['房屋户型'].mode().empty else '2室1厅1卫' # 提供一个备用默认值
    temp_huxing = df_combined['房屋户型'].fillna(huxing_mode)

    # 计算提取后各部分的众数，用于填充提取失败的 NaN
    # 使用 .iloc[0] 来确保即使有多个众数也只取第一个
    # 添加检查，确保 .mode() 不为空
    shi_series = temp_huxing.str.extract(r'(\d+)室', expand=False).astype(float)
    shi_mode = shi_series.mode().iloc[0] if not shi_series.mode().empty else 2.0 # 备用默认值 2室

    ting_series = temp_huxing.str.extract(r'(\d+)厅', expand=False).astype(float)
    ting_mode = ting_series.mode().iloc[0] if not ting_series.mode().empty else 1.0 # 备用默认值 1厅

    wei_series = temp_huxing.str.extract(r'(\d+)卫', expand=False).astype(float)
    wei_mode = wei_series.mode().iloc[0] if not wei_series.mode().empty else 1.0 # 备用默认值 1卫

    # --- 执行提取并填充 ---
    # 使用 .str.extract 并指定 expand=False 返回 Series，然后转换类型和填充
    df_combined['室'] = df_combined['房屋户型'].str.extract(r'(\d+)室', expand=False).astype(float).fillna(shi_mode)
    df_combined['厅'] = df_combined['房屋户型'].str.extract(r'(\d+)厅', expand=False).astype(float).fillna(ting_mode)
    df_combined['卫'] = df_combined['房屋户型'].str.extract(r'(\d+)卫', expand=False).astype(float).fillna(wei_mode)

    # (可选) 处理特殊情况，例如 "房间" 可能需要映射到 '室'，根据需要调整
    # 例如: df_combined.loc[df_combined['房屋户型'].str.contains('房间', na=False), '室'] = df_combined['房屋户型'].str.extract(r'(\d+)房间', expand=False).astype(float).fillna(shi_mode)

    print(f"已生成 '室' (众数填充: {shi_mode}), '厅' (众数填充: {ting_mode}), '卫' (众数填充: {wei_mode}) 特征。")

# (确保后续步骤不再错误地删除 '室', '厅', '卫'，但要删除原始的 '房屋户型')
# 在 STAGE 8 的 original_cols_to_drop 列表中，确保 '房屋户型' 在里面，但 '室', '厅', '卫' 不在里面。


# --- 特征工程: 映射 装修情况 (有序编码) ---
if '装修情况' in df_combined.columns:
    print("--- 正在映射 装修情况 为有序数值... ---")
    map_zhuangxiu = {'毛坯': 0, '简装': 1, '精装': 2, '其他': 1} # 定义映射关系，'其他' 视为 '简装'

    # 确定一个合理的填充值 (例如，使用 '简装'/'其他' 对应的 1 作为默认值)
    # price_model 中也是将缺失值最终处理为 1
    zhuangxiu_default_val = 1

    # 应用映射，并使用 .fillna() 处理原始列中的 NaN 或不在映射字典中的值
    df_combined['装修情况_mapped'] = df_combined['装修情况'].map(map_zhuangxiu).fillna(zhuangxiu_default_val)

    # (可选) 确保结果是整数类型，如果需要的话
    # df_combined['装修情况_mapped'] = df_combined['装修情况_mapped'].astype(int)

    print(f"已生成 '装修情况_mapped' 特征 (缺失值填充为 {zhuangxiu_default_val})。")

# (确保后续步骤不再错误地删除 '装修情况_mapped'，但要删除原始的 '装修情况')
# 在 STAGE 8 的 original_cols_to_drop 列表中，确保 '装修情况' 在里面，但 '装修情况_mapped' 不在里面。

--- STAGE 1: 加载并进行通用预处理... ---
--- 正在转换 Price 为 log_price... ---
--- 正在解析 房屋户型 (提取 室, 厅, 卫)... ---
已生成 '室' (众数填充: 3.0), '厅' (众数填充: 2.0), '卫' (众数填充: 1.0) 特征。
--- 正在映射 装修情况 为有序数值... ---
已生成 '装修情况_mapped' 特征 (缺失值填充为 1)。


In [3]:
# --- STAGE 2: 拆分数据集 (来自 Cell 6) ---
print("\n--- STAGE 2: 拆分数据集... ---")
if 'train' not in df_combined.columns:
    raise KeyError("合并的数据集中必须包含 'train' 指示列 (1=train, 0=test)")

train_df = df_combined[df_combined['train'] == 1].copy()
test_df = df_combined[df_combined['train'] == 0].copy()

train_df = train_df.drop('train', axis=1)
test_df = test_df.drop('train', axis=1)

print(f"原始训练集维度: {train_df.shape}")
print(f"原始测试集维度: {test_df.shape}")


--- STAGE 2: 拆分数据集... ---
原始训练集维度: (103871, 85)
原始测试集维度: (34017, 85)


In [4]:
# --- STAGE 3: 【*新*】 清理训练集 ---
print("\n--- STAGE 3: 【新】 根据您的要求，清理训练集... ---")

# 定义缺失率较低的、必须有值的列 (来自 Cell 7 的 'cols_to_fill_mode_strict')
critical_cols = [
    '房屋户型', '建筑结构', '装修情况', '梯户比例', # 低缺失类别
    'log_建筑面积', # 关键数值
    '总楼层_temp', # 关键数值
    '楼层位置_mapped' # 关键数值
]
# 确保只使用 train_df 中实际存在的列
critical_cols_exist = [col for col in critical_cols if col in train_df.columns]

if critical_cols_exist:
    print(f"--- 正在删除在 {critical_cols_exist} 中包含缺失值的训练行... ---")
    original_train_count = len(train_df)
    train_df_cleaned = train_df.dropna(subset=critical_cols_exist)
    new_train_count = len(train_df_cleaned)
    print(f"--- 训练集行数从 {original_train_count} 减少到 {new_train_count} ---")
else:
    print("--- 未找到指定的关键列，跳过训练集行删除 ---")
    train_df_cleaned = train_df.copy() # 保持一致性

# --- STAGE 4: 仅在【清理后】的训练集上计算所有规则 (来自 Cell 7) ---
print("\n--- STAGE 4: 仅在【清理后】的训练集上计算所有填充和盖帽规则... ---")

# 规则 A: 按板块的众数
cols_to_fill_mode_strict = ['房屋户型', '建筑结构', '装修情况', '梯户比例']
cols_to_fill_mode_other = ['物业类别', '建筑结构_comm', '配备电梯', '房屋朝向', '房屋用途','城市']
cols_to_fill_mode_all = cols_to_fill_mode_strict + cols_to_fill_mode_other

# 计算每个板块的众数
train_df_cleaned['板块'] = train_df_cleaned['板块'].fillna(train_df_cleaned['板块'].mode()[0])
grouped_modes = train_df_cleaned.groupby('板块')[cols_to_fill_mode_all].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0])
global_modes = train_df_cleaned[cols_to_fill_mode_all].mode().iloc[0]

# 规则 B: 中位数
cols_to_fill_median_strict = [
    '建筑年代_parsed', '房屋总数_num', '楼栋总数_num', '绿化率_num', 
    '容积率_num', '燃气费_num', '停车位_num', '停车费用_num', 
    '总楼层_temp', '梯', '户', '楼层位置_mapped'
]
cols_to_fill_median_strict = [col for col in cols_to_fill_median_strict if col in train_df_cleaned.columns]


--- STAGE 3: 【新】 根据您的要求，清理训练集... ---
--- 正在删除在 ['房屋户型', '建筑结构', '装修情况', '梯户比例', 'log_建筑面积', '总楼层_temp', '楼层位置_mapped'] 中包含缺失值的训练行... ---
--- 训练集行数从 103871 减少到 101252 ---

--- STAGE 4: 仅在【清理后】的训练集上计算所有填充和盖帽规则... ---


In [5]:
# 【修改】使用 train_df_cleaned
# 填充 '板块' 以便分组 (仅在清理后的训练集上)
train_df_cleaned['板块'] = train_df_cleaned['板块'].fillna(train_df_cleaned['板块'].mode()[0]) 
train_bankuai_mode = train_df_cleaned['板块'].mode()[0] # 规则

# 【修改】使用 train_df_cleaned
grouped_medians = train_df_cleaned.groupby('板块')[cols_to_fill_median_strict].median()
global_medians = train_df_cleaned[cols_to_fill_median_strict].median()

# 规则 C: 盖帽值
cap_values = {}
# 【修改】使用 train_df_cleaned
if '持有天数' in train_df_cleaned.columns: cap_values['持有天数_lower'] = 0 
if '总楼层_temp' in train_df_cleaned.columns: cap_values['总楼层_lower'] = 1 
if '绿化率_num' in train_df_cleaned.columns: cap_values['绿化率_num_upper'] = 1.0 
if '容积率_num' in train_df_cleaned.columns: cap_values['容积率_num_upper'] = train_df_cleaned['容积率_num'].quantile(0.99)
if '户' in train_df_cleaned.columns:
    p_99_hu_train = train_df_cleaned['户'].quantile(0.99)
    cap_values['户_upper'] = min(p_99_hu_train if pd.notna(p_99_hu_train) else 50, 50) 
if '梯' in train_df_cleaned.columns:
    p_99_ti_train = train_df_cleaned['梯'].quantile(0.99)
    cap_values['梯_upper'] = min(p_99_ti_train if pd.notna(p_99_ti_train) else 20, 20)

# 规则 D: K-Means 拟合
kmeans_model = None
coord_cols = []
# 【修改】使用 train_df_cleaned
if 'lon' in train_df_cleaned.columns and 'lat' in train_df_cleaned.columns and train_df_cleaned['lon'].notna().all() and train_df_cleaned['lat'].notna().all():
    coord_cols = ['lon', 'lat']
elif 'coord_x' in train_df_cleaned.columns and 'coord_y' in train_df_cleaned.columns:
     train_df_cleaned['coord_x'] = train_df_cleaned['coord_x'].fillna(train_df_cleaned['coord_x'].median())
     train_df_cleaned['coord_y'] = train_df_cleaned['coord_y'].fillna(train_df_cleaned['coord_y'].median())
     coord_cols = ['coord_x', 'coord_y']

# --- 新增：确定最优K-Means聚类数量 ---
if coord_cols:
    print(f"--- 正在确定最优K-Means聚类数量（使用 {coord_cols}）... ---")
    
    from sklearn.metrics import silhouette_score
    
    # 准备坐标数据
    coords = train_df_cleaned[coord_cols].values
    
    # 尝试不同的聚类数量
    k_range = range(3, 12)  # 尝试3到11个聚类
    wcss = []  # 簇内平方和
    
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=111, n_init=10)
        kmeans.fit(coords)
        wcss.append(kmeans.inertia_)
    
    # 使用肘部法则确定最优k
    # 计算每个k对应的斜率变化（二阶导数）
    wcss_diff = [wcss[i-1] - wcss[i] for i in range(1, len(wcss))]
    wcss_diff_diff = [wcss_diff[i-1] - wcss_diff[i] for i in range(1, len(wcss_diff))]
    
    # 找到斜率变化最大的点（肘部）
    optimal_k = k_range[wcss_diff_diff.index(max(wcss_diff_diff)) + 2]
    print(f"--- 最优聚类数量: {optimal_k} ---")
    
    # 使用最优k训练最终模型
    kmeans_model = KMeans(n_clusters=10, random_state=111, n_init=10)
    kmeans_model.fit(train_df_cleaned[coord_cols])
else:
    print("警告: 未找到合适的坐标列 (lon/lat 或 coord_x/y) 用于 KMeans。")
    kmeans_model = None

print("--- 所有规则已计算完毕 ---")


print("\n--- 正在计算 *早期可用特征* 的分箱边界 (基于 train_df_cleaned)... ---")

bin_edges = {} # 用于存储所有特征的分箱边界
# 只包含 STAGE 4 时已存在的特征
features_to_bin_early = ['log_建筑面积']
# 稍后在 apply_rules 计算后处理的特征
features_to_bin_late = ['房龄', '持有天数']
n_bins = 6 # 定义箱数

# --- 计算早期特征的边界 ---
for feature in features_to_bin_early:
    if feature in train_df_cleaned.columns:
        try:
            _, edges = pd.qcut(train_df_cleaned[feature], q=n_bins, labels=False, retbins=True, duplicates='drop')
            bin_edges[feature] = np.unique(edges) # 存储边界
            print(f"为 '{feature}' 计算了 {len(bin_edges[feature])-1} 个分箱的边界: {np.round(bin_edges[feature], 2)}")
        except Exception as e:
            print(f"为 '{feature}' 计算分箱边界时出错: {e}. 跳过此特征。")
            if feature in bin_edges: del bin_edges[feature]
    else:
        print(f"警告: 特征 '{feature}' 不在 train_df_cleaned 中，无法进行分箱。")

print(f"--- 早期分箱边界计算完毕。将在 apply_rules 应用后计算 {features_to_bin_late} 的边界。 ---")
# --- >>> 修改代码结束 <<< ---

--- 正在确定最优K-Means聚类数量（使用 ['lon', 'lat']）... ---
--- 最优聚类数量: 5 ---
--- 所有规则已计算完毕 ---

--- 正在计算 *早期可用特征* 的分箱边界 (基于 train_df_cleaned)... ---
为 'log_建筑面积' 计算了 6 个分箱的边界: [2.62 4.12 4.39 4.52 4.67 4.87 6.22]
--- 早期分箱边界计算完毕。将在 apply_rules 应用后计算 ['房龄', '持有天数'] 的边界。 ---


In [6]:

# --- STAGE 5: 定义应用规则的函数 (来自 Cell 8 - *已修正结构*) ---
# (这个函数现在至关重要，它将用于填充 test_df 和 train_df_cleaned 中的剩余高缺失列)
print("\n--- STAGE 5: 定义应用规则的函数... ---")

def apply_rules(df, is_test_set=False):
    df_processed = df.copy()
    
    # 确保板块没有缺失值
    df_processed['板块'] = df_processed['板块'].fillna(train_bankuai_mode)
    
    # 规则 A: 按板块众数填充分类变量
    df_processed = df_processed.merge(grouped_modes, on='板块', how='left', suffixes=('', '_train_mode'))
    
    for col in cols_to_fill_mode_all:
        mode_col_name = f'{col}_train_mode'
        df_processed[col] = df_processed[col].fillna(df_processed[mode_col_name])
        df_processed[col] = df_processed[col].fillna(global_modes[col])
    
    # 删除辅助列
    helper_cols_mode = [f'{col}_train_mode' for col in cols_to_fill_mode_all]
    df_processed = df_processed.drop(columns=helper_cols_mode, errors='ignore')
    
# --- >>> 新增代码开始: 应用分箱 <<< ---
    print("--- (apply_rules) 正在应用分箱... ---")
    global bin_edges # 确保能访问到 STAGE 4 计算的边界

    for feature, edges in bin_edges.items():
        if feature in df_processed.columns:
            binned_col_name = f'{feature}_binned'
            try:
                # 使用 pd.cut 应用分箱边界
                # include_lowest=True 包含最小值
                # labels=False 返回整数标签 (0, 1, 2, ...)
                df_processed[binned_col_name] = pd.cut(df_processed[feature], bins=edges, labels=False, include_lowest=True)
                # 将 NaN 填充为一个特殊的类别（例如 -1），或者用众数填充
                # 这里选择填充 -1，表示缺失或超出边界
                df_processed[binned_col_name] = df_processed[binned_col_name].fillna(-1).astype(int)
                print(f"已生成分箱特征: {binned_col_name}")
            except Exception as e:
                print(f"应用分箱到 '{feature}' 时出错: {e}")
        else:
             print(f"--- (apply_rules) 警告: 特征 '{feature}' 不在待处理数据框中，无法应用分箱。---")
    # --- >>> 新增代码结束 <<< ---


    
# --- >>> 新增代码开始 <<< ---
    # --- 在填充 '配备电梯' 之后，进行二元映射 ---
    if '配备电梯' in df_processed.columns:
        print("--- 正在映射 '配备电梯' 为 0/1 ... ---")
        # 此时 '配备电梯' 列应该已经被上面的循环填充完毕
        df_processed['配备电梯_mapped'] = df_processed['配备电梯'].map({'有': 1, '无': 0})

        # 添加一个额外的 .fillna(0) 作为保险，处理极其罕见的填充失败或原始值不是'有'/'无'的情况
        # 假设没有电梯 (0) 是更安全的默认值
        if df_processed['配备电梯_mapped'].isnull().any():
             print("警告: '配备电梯_mapped' 中发现 NaN，将用 0 填充。")
             df_processed['配备电梯_mapped'] = df_processed['配备电梯_mapped'].fillna(0)

        # 确保是整数类型
        df_processed['配备电梯_mapped'] = df_processed['配备电梯_mapped'].astype(int)

    
    # 规则 B: 按板块中位数填充数值变量（保持不变）
    df_processed = df_processed.merge(grouped_medians, on='板块', how='left', suffixes=('', '_train_median'))
    
    for col in cols_to_fill_median_strict:
        median_col_name = f'{col}_train_median'
        df_processed[col] = df_processed[col].fillna(df_processed[median_col_name])
        df_processed[col] = df_processed[col].fillna(global_medians[col])
    
    helper_cols_median = [f'{col}_train_median' for col in cols_to_fill_median_strict]
    df_processed = df_processed.drop(columns=helper_cols_median, errors='ignore')

    
    if '总楼层_temp' in df_processed.columns:
        df_processed.rename(columns={'总楼层_temp': '总楼层'}, inplace=True)

    # 计算 '梯户比' 和 '楼层相对位置' (在填充后)
    if '梯' in df_processed.columns and '户' in df_processed.columns:
        df_processed['户'] = df_processed['户'].replace(0, 2) 
        df_processed['梯户比'] = df_processed['梯'] / df_processed['户']
    if '楼层位置_mapped' in df_processed.columns and '总楼层' in df_processed.columns:
        df_processed['总楼层'] = df_processed['总楼层'].clip(lower=1) 
        df_processed['楼层相对位置'] = df_processed['楼层位置_mapped'] / df_processed['总楼层']

    # 计算 '房龄'
    if '建筑年代_parsed' in df_processed.columns:
        df_processed['房龄'] = 2025 - df_processed['建筑年代_parsed']

    # --- 多标签二值化 (建筑结构_comm) ---
    if '建筑结构_comm' in df_processed.columns:
        main_types_jiegou_comm = ['塔楼', '板楼', '塔板结合', '平房']
        for j_type in main_types_jiegou_comm:
            df_processed[f'建筑结构_comm_{j_type}'] = df_processed['建筑结构_comm'].astype(str).str.contains(j_type, na=False).astype(int)
            
    # --- 特征工程: 多标签 (物业类别) ---
    if '物业类别' in df_processed.columns:
        main_types = ['普通住宅', '别墅', '写字楼', '商业', '公寓', '底商', '车库', '花园洋房', '平房', '新式里弄', '老公寓']
        for prop_type in main_types:
            df_processed[f'物业类型_{prop_type}'] = df_processed['物业类别'].astype(str).str.contains(prop_type, na=False).astype(int)
    
    # --- 特征工程: 多标签 (房屋用途) ---
    if '房屋用途' in df_processed.columns:
            main_types_yongtu = [
                '普通住宅', '别墅', '商业办公类', '车库', '公寓', '酒店式公寓', 
                '四合院', '商务型公寓', '住宅式公寓', '商住两用', '新式里弄', 
                '老公寓', '花园洋房', '底商', '商业', '商务公寓', '写字楼', '住宅'
            ]
            for y_type in main_types_yongtu:
                df_processed[f'房屋用途_{y_type}'] = df_processed['房屋用途'].astype(str).str.contains(y_type, na=False).astype(int)

    # (!!! 以下是您 Cell 8 中在函数定义之外的逻辑，已移入函数内 !!!)
    
    # 规则 C: 盖帽
    if '持有天数' in df_processed.columns: df_processed['持有天数'] = df_processed['持有天数'].clip(lower=cap_values.get('持有天数_lower'))
    if '总楼层' in df_processed.columns: df_processed['总楼层'] = df_processed['总楼层'].clip(lower=cap_values.get('总楼层_lower'))
    if '绿化率_num' in df_processed.columns: df_processed['绿化率_num'] = df_processed['绿化率_num'].clip(upper=cap_values.get('绿化率_num_upper'))
    if '容积率_num' in df_processed.columns: df_processed['容积率_num'] = df_processed['容积率_num'].clip(upper=cap_values.get('容积率_num_upper'))
    if '户' in df_processed.columns: df_processed['户'] = df_processed['户'].clip(upper=cap_values.get('户_upper'))
    if '梯' in df_processed.columns: df_processed['梯'] = df_processed['梯'].clip(upper=cap_values.get('梯_upper'))
    
    if '梯户比' in df_processed.columns:
         p_99_bihu_train_cap = cap_values.get('梯户比_upper')
         if p_99_bihu_train_cap is not None:
             df_processed['梯户比'] = df_processed['梯户比'].clip(upper=p_99_bihu_train_cap)

    # 规则 D: K-Means 预测
    if kmeans_model is not None and coord_cols:
        for c_col in coord_cols:
             if c_col not in df_processed.columns or df_processed[c_col].isnull().any():
                  # (使用全局中位数填充坐标，以防万一)
                  coord_median = global_medians.get(c_col, df_processed[c_col].median())
                  df_processed[c_col] = df_processed[c_col].fillna(coord_median)
        df_processed['location_cluster'] = kmeans_model.predict(df_processed[coord_cols])
        df_processed['location_cluster'] = df_processed['location_cluster'].astype('category')

    # 特征工程: 日期
    if '交易时间' in df_processed.columns:
        df_processed['交易年份'] = df_processed['交易时间'].dt.year
        df_processed['交易月份'] = df_processed['交易时间'].dt.month
    if '交易时间' in df_processed.columns and '上次交易' in df_processed.columns:
         df_processed['持有天数'] = (df_processed['交易时间'] - df_processed['上次交易']).dt.days
         df_processed['是否首次交易'] = df_processed['上次交易'].isnull().astype(int)
         df_processed['持有天数'] = df_processed['持有天数'].fillna(0)
         if '持有天数_lower' in cap_values:
             df_processed['持有天数'] = df_processed['持有天数'].clip(lower=cap_values['持有天数_lower'])

    df_processed = df_processed.drop(['coord_x', 'coord_y'], axis=1, errors='ignore')

    return df_processed



--- STAGE 5: 定义应用规则的函数... ---


In [7]:
# --- >>> 新增代码开始: 计算后期特征的分箱边界 <<< ---
print("\n--- 正在计算 *后期生成特征* 的分箱边界... ---")
print("--- (需要先对 train_df_cleaned 应用一次 apply_rules 以生成特征) ---")

# 临时应用 apply_rules 以获得包含 '房龄' 和 '持有天数' 的训练集
# 注意：这里调用 apply_rules 主要是为了生成列，里面的分箱应用会因为 bin_edges 不全而部分跳过，没关系
temp_train_processed_for_bins = apply_rules(train_df_cleaned, is_test_set=False)

# 现在 temp_train_processed_for_bins 中应该有 '房龄' 和 '持有天数' 了
for feature in features_to_bin_late: # 使用之前定义的后期特征列表
    if feature in temp_train_processed_for_bins.columns:
        try:
            # 确保数据无 NaN (apply_rules 应该已经处理了，这里是保险)
            feature_data = temp_train_processed_for_bins[feature].dropna()
            if not feature_data.empty:
                _, edges = pd.qcut(feature_data, q=n_bins, labels=False, retbins=True, duplicates='drop')
                bin_edges[feature] = np.unique(edges) # 更新 bin_edges 字典
                print(f"为 '{feature}' 计算了 {len(bin_edges[feature])-1} 个分箱的边界: {np.round(bin_edges[feature], 2)}")
            else:
                print(f"警告: 特征 '{feature}' 数据为空或全是 NaN，无法计算分箱边界。")
        except Exception as e:
            print(f"为 '{feature}' 计算分箱边界时出错: {e}. 跳过此特征。")
            if feature in bin_edges: del bin_edges[feature]
    else:
        print(f"警告: 特征 '{feature}' 未在临时处理的训练集中生成，无法计算分箱边界。")

# 清理临时变量 (可选)
del temp_train_processed_for_bins

print("--- 所有分箱边界计算完毕 ---")
print("--- 完整的 bin_edges 字典:", {k: np.round(v, 2).tolist() for k, v in bin_edges.items()}) # 打印确认
# --- >>> 新增代码结束 <<< ---

# (紧接着是 STAGE 6 的代码)


--- 正在计算 *后期生成特征* 的分箱边界... ---
--- (需要先对 train_df_cleaned 应用一次 apply_rules 以生成特征) ---
--- (apply_rules) 正在应用分箱... ---
已生成分箱特征: log_建筑面积_binned
--- 正在映射 '配备电梯' 为 0/1 ... ---
为 '房龄' 计算了 5 个分箱的边界: [ 3.5 11.  14.5 17.  23.  92. ]
为 '持有天数' 计算了 6 个分箱的边界: [-31098.        0.     1259.     2011.     2854.     4383.83  19790.  ]
--- 所有分箱边界计算完毕 ---
--- 完整的 bin_edges 字典: {'log_建筑面积': [2.62, 4.12, 4.39, 4.52, 4.67, 4.87, 6.22], '房龄': [3.5, 11.0, 14.5, 17.0, 23.0, 92.0], '持有天数': [-31098.0, 0.0, 1259.0, 2011.0, 2854.0, 4383.83, 19790.0]}


In [8]:

# --- STAGE 6: 应用规则到【清理后】的训练集和【原始】测试集 (来自 Cell 9) ---
print("\n--- STAGE 6: 将规则应用于数据集... ---")

# 【修改】对 train_df_cleaned 应用规则
# (这将填充剩余的高缺失率列，如 容积率_num)
print("--- 正在处理 train_df_cleaned... ---")
train_df_processed = apply_rules(train_df_cleaned, is_test_set=False)

# 【修改】对 test_df 应用规则
# (这将填充所有缺失列，包括低缺失和高缺失)
print("--- 正在处理 test_df... ---")
test_df_processed = apply_rules(test_df, is_test_set=True)


# --- STAGE 7: One-Hot 编码 (确保列对齐) (来自 Cell 9) ---
print("\n--- STAGE 7: 执行 One-Hot 编码并对齐列... ---")

categorical_cols = [
    '建筑结构', '交易权属', '产权所属', '供水', '供电', 'location_cluster','区域'
]

# --- >>> 新增代码开始: 添加分箱后的列 <<< ---
# 根据你在 features_to_bin 中选择的特征动态添加
binned_cols_to_encode = [f'{feature}_binned' for feature in bin_edges.keys()]
print(f"--- 将添加以下分箱列进行独热编码: {binned_cols_to_encode} ---")
# --- >>> 新增代码结束 <<< ---

categorical_cols = [col for col in categorical_cols if col in train_df_processed.columns] 

train_df_encoded = pd.get_dummies(train_df_processed, columns=categorical_cols, dummy_na=False)
test_df_encoded = pd.get_dummies(test_df_processed, columns=categorical_cols, dummy_na=False)

# (关键) 列对齐
train_cols = train_df_encoded.drop('log_price', axis=1, errors='ignore').columns 
test_cols = test_df_encoded.columns

cols_to_drop_from_test = list(set(test_cols) - set(train_cols) - set(['ID'])) 
test_df_aligned = test_df_encoded.drop(columns=cols_to_drop_from_test, errors='ignore')

cols_to_add_to_test = list(set(train_cols) - set(test_cols))
for col in cols_to_add_to_test:
    test_df_aligned[col] = 0

# 确保测试集的列顺序与训练集一致 (去掉 log_price)
# (如果ID在train_cols中，需要去掉)
train_cols_final_order = [col for col in train_cols if col != 'ID']
test_df_aligned = test_df_aligned[train_cols_final_order] # 应用训练集顺序




--- STAGE 6: 将规则应用于数据集... ---
--- 正在处理 train_df_cleaned... ---
--- (apply_rules) 正在应用分箱... ---
已生成分箱特征: log_建筑面积_binned
--- (apply_rules) 警告: 特征 '房龄' 不在待处理数据框中，无法应用分箱。---
--- (apply_rules) 警告: 特征 '持有天数' 不在待处理数据框中，无法应用分箱。---
--- 正在映射 '配备电梯' 为 0/1 ... ---
--- 正在处理 test_df... ---
--- (apply_rules) 正在应用分箱... ---
已生成分箱特征: log_建筑面积_binned
--- (apply_rules) 警告: 特征 '房龄' 不在待处理数据框中，无法应用分箱。---
--- (apply_rules) 警告: 特征 '持有天数' 不在待处理数据框中，无法应用分箱。---
--- 正在映射 '配备电梯' 为 0/1 ... ---

--- STAGE 7: 执行 One-Hot 编码并对齐列... ---
--- 将添加以下分箱列进行独热编码: ['log_建筑面积_binned', '房龄_binned', '持有天数_binned'] ---


In [9]:
# --- STAGE 8: 最终清理 (来自 Cell 10) ---
print("\n--- STAGE 8: 最终清理... ---")

original_cols_to_drop = [
    '房屋户型', '装修情况', '梯户比例', '配备电梯', '所在楼层', '房屋朝向',
    '建筑年代', '房屋总数', '楼栋总数', '绿 化 率', '容 积 率', '燃气费', 
    '停车位', '停车费用', '交易时间', '上次交易', '房屋用途',
    '房屋优势', '周边配套', '交通出行', '核心卖点', '客户反馈', 
    '板块', '物业类别', '建筑结构', '交易权属', '房屋用途', '产权所属', 
    '建筑结构_comm', '供水', '供电', '区域', 'location_cluster', 
    '建筑年代_parsed', 'Price', '抵押信息', '区县', '板块_comm', 
    'coord_x', 'coord_y','城市'
]
original_cols_to_drop = list(set(original_cols_to_drop))

train_df_final = train_df_encoded.drop(columns=original_cols_to_drop, errors='ignore')
test_df_final = test_df_aligned.drop(columns=original_cols_to_drop, errors='ignore')

# 删除所有剩余的 object 类型
train_obj_cols = train_df_final.select_dtypes(include=['object']).columns
test_obj_cols = test_df_final.select_dtypes(include=['object']).columns
train_df_final = train_df_final.drop(columns=train_obj_cols, errors='ignore')
test_df_final = test_df_final.drop(columns=test_obj_cols, errors='ignore')

# 从最终训练集中移除 ID 列
if 'ID' in train_df_final.columns:
    train_df_final = train_df_final.drop('ID', axis=1, errors='ignore')

# (确保 ID 仍然在最终测试集中)
if 'ID' not in test_df_final.columns and 'ID' in test_df.columns:
     test_df_final['ID'] = test_df['ID']


# --- >>> 新增代码开始 <<< ---
# --- 移除零方差特征 ---
print("\n--- 正在检查并移除零方差特征... ---")

# 1. 准备训练特征集 (不含目标变量 'log_price')
X_train_features = train_df_final.drop('log_price', axis=1, errors='ignore')

# 2. 计算训练特征集的方差
try:
    variances = X_train_features.var()

    # 3. 找出方差为 0 (或非常接近 0) 的列
    #    使用一个小的阈值来处理可能的浮点数精度问题
    zero_variance_cols = variances[variances <= 1e-8].index.tolist()

    if zero_variance_cols:
        print(f"找到以下零方差特征: {zero_variance_cols}")

        # 4. 从训练集和测试集中移除这些列
        #    确保 'log_price' 不会被误删 (虽然理论上它方差不为0)
        cols_to_drop_final_train = [col for col in zero_variance_cols if col in train_df_final.columns and col != 'log_price']
        #    确保 'ID' 不会被误删
        cols_to_drop_final_test = [col for col in zero_variance_cols if col in test_df_final.columns and col != 'ID']

        if cols_to_drop_final_train:
            train_df_final = train_df_final.drop(columns=cols_to_drop_final_train)
            print(f"已从 train_df_final 中移除 {len(cols_to_drop_final_train)} 个零方差列。")

        if cols_to_drop_final_test:
            test_df_final = test_df_final.drop(columns=cols_to_drop_final_test)
            print(f"已从 test_df_final 中移除 {len(cols_to_drop_final_test)} 个零方差列。")

        # 再次打印最终维度
        print(f"移除零方差特征后，最终训练集维度: {train_df_final.shape}")
        print(f"移除零方差特征后，最终测试集维度: {test_df_final.shape}")

        # 再次验证列匹配
        train_features_after_drop = set(train_df_final.drop('log_price', axis=1, errors='ignore').columns)
        test_features_after_drop = set(test_df_final.drop('ID', axis=1, errors='ignore').columns)
        if train_features_after_drop == test_features_after_drop:
            print(f"移除后列匹配成功！特征数量: {len(train_features_after_drop)}")
        else:
            print("警告：移除零方差列后，列仍然不匹配！请检查。")

    else:
        print("未在训练集中找到零方差特征。")

except Exception as e:
    print(f"计算或移除零方差特征时出错: {e}")
    print("跳过零方差特征移除步骤。")

# --- >>> 新增代码结束 <<< ---

# (后续的代码，如 STAGE 9 模型训练，将使用更新后的 train_df_final 和 test_df_final)


--- STAGE 8: 最终清理... ---

--- 正在检查并移除零方差特征... ---
找到以下零方差特征: ['有_客户反馈', '房屋用途_车库']
已从 train_df_final 中移除 2 个零方差列。
已从 test_df_final 中移除 2 个零方差列。
移除零方差特征后，最终训练集维度: (101252, 225)
移除零方差特征后，最终测试集维度: (34017, 225)
移除后列匹配成功！特征数量: 224


In [10]:
# --- 最终结果 ---
print("\n--- 处理完成 ---")
print(f"最终训练集维度: {train_df_final.shape}")
print(f"最终测试集维度: {test_df_final.shape}")

# 验证列是否匹配 (除了 log_price 和 ID)
train_features = set(train_df_final.drop('log_price', axis=1, errors='ignore').columns)
test_features = set(test_df_final.drop('ID', axis=1, errors='ignore').columns)

if train_features == test_features:
    print(f"列匹配成功！特征数量: {len(train_features)}")
else:
    print("警告：列不匹配！")
    print("仅在训练集中的列:", list(train_features - test_features))
    print("仅在测试集中的列:", list(test_features - train_features))

# 打印最终训练集的列 (来自 Cell 11)
print("\n--- 最终训练集 (train_df_final) 的列 ---")
all_columns = train_df_final.columns.tolist()
for i, col in enumerate(all_columns, 1):
    print(f"{i:3d}. {col}")

# 打印最终测试集的列
print("\n--- 最终测试集 (test_df_final) 的列 ---")
all_columns_test = test_df_final.columns.tolist()
for i, col in enumerate(all_columns_test, 1):
    print(f"{i:3d}. {col}")


--- 处理完成 ---
最终训练集维度: (101252, 225)
最终测试集维度: (34017, 225)
列匹配成功！特征数量: 224

--- 最终训练集 (train_df_final) 的列 ---
  1. lon
  2. lat
  3. 年份
  4. log_price
  5. 供水_商水
  6. 供水_民水
  7. 供电_商电
  8. 供电_民电
  9. log_建筑面积
 10. 梯
 11. 户
 12. 总楼层
 13. 楼层位置_mapped
 14. 朝南
 15. 朝北
 16. 朝东
 17. 朝西
 18. 房屋总数_num
 19. 楼栋总数_num
 20. 绿化率_num
 21. 容积率_num
 22. 燃气费_num
 23. 停车位_num
 24. 停车费用_num
 25. 有_房屋优势
 26. 有_周边配套
 27. 有_交通出行
 28. 有_核心卖点
 29. 核心卖点_长度
 30. 室
 31. 厅
 32. 卫
 33. 装修情况_mapped
 34. log_建筑面积_binned
 35. 配备电梯_mapped
 36. 梯户比
 37. 楼层相对位置
 38. 房龄
 39. 建筑结构_comm_塔楼
 40. 建筑结构_comm_板楼
 41. 建筑结构_comm_塔板结合
 42. 建筑结构_comm_平房
 43. 物业类型_普通住宅
 44. 物业类型_别墅
 45. 物业类型_写字楼
 46. 物业类型_商业
 47. 物业类型_公寓
 48. 物业类型_底商
 49. 物业类型_车库
 50. 物业类型_花园洋房
 51. 物业类型_平房
 52. 物业类型_新式里弄
 53. 物业类型_老公寓
 54. 房屋用途_普通住宅
 55. 房屋用途_别墅
 56. 房屋用途_商业办公类
 57. 房屋用途_公寓
 58. 房屋用途_酒店式公寓
 59. 房屋用途_四合院
 60. 房屋用途_商务型公寓
 61. 房屋用途_住宅式公寓
 62. 房屋用途_商住两用
 63. 房屋用途_新式里弄
 64. 房屋用途_老公寓
 65. 房屋用途_花园洋房
 66. 房屋用途_底商
 67. 房屋用途_商业
 68. 房屋用途_商务公寓
 69. 房屋用途_写字楼
 

In [11]:
# --- >>> 新增代码开始: 检查核心数值特征的相关系数 <<< ---
print("\n--- 正在计算核心数值特征的相关系数矩阵... ---")

# 选择你认为重要的、非独热编码产生的数值特征
# (这是一个示例列表，你需要根据你的最终特征集调整)
core_numerical_features = [
    # --- 地理位置与时间 ---
    'lon',                 # 经度 (连续)
    'lat',                 # 纬度 (连续)
    '年份',                # 原始数据中的年份 (离散/有序) - 注意：可能与 '交易年份' 相关
    '交易年份',            # 从交易时间解析 (离散/有序)
    '交易月份',            # 从交易时间解析 (离散/有序)

    # --- 房屋基本属性 ---
    'log_建筑面积',        # 对数转换后的建筑面积 (连续)
    '室',                  # 房间数 (离散计数)
    '厅',                  # 客厅数 (离散计数)
    '卫',                  # 卫生间数 (离散计数)
    '装修情况_mapped',     # 装修情况的有序映射 (0, 1, 2) - 如果你添加了
    '配备电梯_mapped',     # 是否有电梯的二元映射 (0, 1) - 如果你添加了

    # --- 楼层与结构相关 ---
    '梯',                  # 电梯数量 (离散计数)
    '户',                  # 每层户数 (离散计数)
    '梯户比',              # 计算得到的比率 (连续)
    '总楼层',              # 总楼层数 (离散计数)
    '楼层位置_mapped',     # 楼层位置的有序映射 (-1 到 5)
    '楼层相对位置',        # 计算得到的比率 (连续)

    # --- 朝向 (二元) ---
    '朝南',
    '朝北',
    '朝东',
    '朝西',

    # --- 社区/建筑属性 ---
    '房屋总数_num',        # 小区房屋总数 (离散计数)
    '楼栋总数_num',        # 小区楼栋总数 (离散计数)
    '绿化率_num',          # 绿化率 (连续, 0-1)
    '容积率_num',          # 容积率 (连续)
    '房龄',                # 计算得到的房龄 (连续)

    # --- 费用与交易相关 ---
    '燃气费_num',          # 燃气费单价 (连续)
    '停车位_num',          # 停车位数量 (离散计数)
    '停车费用_num',        # 停车费用 (连续)
    '持有天数',            # 房屋持有时间 (连续)
    '是否首次交易',        # 是否为首次交易 (二元 0/1)

    # --- 文本长度 ---
    '核心卖点_长度',       # 核心卖点文本长度 (离散计数)

    # --- 水电 (二元) ---
    '供水_商水',
    '供水_民水',
    '供电_商电',
    '供电_民电',

    # --- (可选) 添加少量关键的多标签二值化特征 ---
    # 如果你认为某几个物业类型或房屋用途特别重要，可以加进来看看
    # 例如:
    # '物业类型_别墅',
    # '物业类型_普通住宅',
    # '房屋用途_普通住宅', # 注意与物业类型的重叠
]

# 在你的代码中，使用这个列表前，务必用下面的代码确保所有列都实际存在于你的 DataFrame 中：
# core_numerical_features_exist = [col for col in core_numerical_features if col in train_df_final.columns]
# 确保这些列在最终的训练集中存在
core_numerical_features_exist = [col for col in core_numerical_features if col in train_df_final.columns]

if core_numerical_features_exist:
    # 仅在训练集上计算
    correlation_matrix = train_df_final[core_numerical_features_exist].corr()

    # 打印出相关系数绝对值大于某个阈值 (例如 0.8) 的特征对
    high_corr_threshold = 0.8
    highly_correlated_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > high_corr_threshold:
                pair = (correlation_matrix.columns[i], correlation_matrix.columns[j], correlation_matrix.iloc[i, j])
                highly_correlated_pairs.append(pair)
                print(f"高度相关: {pair[0]} 和 {pair[1]}, 相关系数: {pair[2]:.3f}")

    if not highly_correlated_pairs:
        print(f"在核心数值特征中未发现绝对值大于 {high_corr_threshold} 的相关系数。")

    # (可选) 可视化热力图 (需要导入 matplotlib 和 seaborn)
    # import matplotlib.pyplot as plt
    # import seaborn as sns
    # plt.figure(figsize=(12, 10)) # 可能需要调整大小
    # sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', fmt=".2f")
    # plt.title('核心数值特征相关系数热力图')
    # plt.show() # 在 Jupyter 环境中显示

else:
    print("未找到用于计算相关系数的核心数值特征列。")

# --- >>> 新增代码结束 <<< ---


--- 正在计算核心数值特征的相关系数矩阵... ---
高度相关: 交易年份 和 年份, 相关系数: 0.887
高度相关: 供电_商电 和 供水_商水, 相关系数: 0.905
高度相关: 供电_民电 和 供水_民水, 相关系数: 0.990


In [12]:
# --- >>> 新增代码开始: 计算核心数值特征的 VIF <<< ---
print("\n--- 正在计算核心数值特征的方差膨胀因子 (VIF)... ---")

# 需要导入库
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import pandas as pd

# 同样使用上面定义的 core_numerical_features_exist 列表
if core_numerical_features_exist:
    # 1. 选择核心数值特征数据 (仅训练集)
    X_vif_check = train_df_final[core_numerical_features_exist].copy()

    # 2. 处理可能的无穷大或缺失值 (VIF 计算前需要)
    X_vif_check = X_vif_check.replace([np.inf, -np.inf], np.nan) # 将无穷大替换为 NaN
    if X_vif_check.isnull().any().any():
        print("警告: 用于 VIF 计算的数据中存在 NaN，将用中位数填充。")
        X_vif_check = X_vif_check.fillna(X_vif_check.median()) # 用中位数填充 NaN

    # 3. 添加常数项 (截距) - VIF 计算需要
    X_vif_check_const = add_constant(X_vif_check)

    # 4. 计算 VIF
    vif_data = pd.DataFrame()
    vif_data["feature"] = X_vif_check_const.columns
    # 逐个计算 VIF
    try:
        vif_data["VIF"] = [variance_inflation_factor(X_vif_check_const.values, i)
                           for i in range(X_vif_check_const.shape[1])]
    except Exception as e:
         print(f"计算VIF时出错（可能是完全共线性）: {e}")
         # 如果出错，可以尝试移除导致问题的列或简化特征集

    # 5. 显示 VIF 结果 (排除常数项 'const')
    vif_results = vif_data[vif_data["feature"] != "const"].sort_values(by="VIF", ascending=False)
    print("\nVIF 计算结果 (降序):")
    print(vif_results)

    # 识别高 VIF 值
    high_vif_threshold = 10 # 通常选择 5 或 10 作为阈值
    high_vif_features = vif_results[vif_results["VIF"] > high_vif_threshold]["feature"].tolist()

    if high_vif_features:
        print(f"\n发现 VIF > {high_vif_threshold} 的特征: {high_vif_features}")
        print("建议考虑处理这些特征（例如，移除其中一个或进行组合）。")
    else:
        print(f"\n未发现 VIF > {high_vif_threshold} 的核心数值特征。")

else:
    print("未找到用于计算 VIF 的核心数值特征列。")

# --- >>> 新增代码结束 <<< ---


--- 正在计算核心数值特征的方差膨胀因子 (VIF)... ---

VIF 计算结果 (降序):
        feature        VIF
36        供电_民电  50.479866
34        供水_民水  50.416182
4          交易年份   8.350622
3            年份   8.273602
33        供水_商水   5.930448
35        供电_商电   5.845839
6      log_建筑面积   4.181203
17       楼层相对位置   4.038084
15          总楼层   3.974575
1           lon   3.831563
12            梯   3.794161
7             室   3.311338
13            户   3.099911
2           lat   3.036556
27      燃气费_num   2.466156
11  配备电梯_mapped   2.328370
9             卫   2.187537
31       是否首次交易   1.988789
30         持有天数   1.898004
5          交易月份   1.829683
26           房龄   1.820969
29     停车费用_num   1.686970
22     房屋总数_num   1.684611
8             厅   1.682716
14          梯户比   1.649626
18           朝南   1.527906
28      停车位_num   1.507909
23     楼栋总数_num   1.498925
16  楼层位置_mapped   1.466172
20           朝东   1.391745
19           朝北   1.391328
25      容积率_num   1.258397
21           朝西   1.227676
24      绿化率_num   1.196067
32 

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler # 仅需要 StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.pipeline import Pipeline 
import warnings

warnings.filterwarnings('ignore')

# 假设 train_df_final 和 test_df_final 是您上一步预处理代码的输出
# train_df_final: 包含 log_price, (N_train_cleaned, N_features + 1)
# test_df_final: 包含 ID, (N_test, N_features + 1)

print("--- 1. 准备训练和测试数据 ---")

# a. 分离训练集的 X 和 y
if 'log_price' in train_df_final.columns:
    X_train_full = train_df_final.drop('log_price', axis=1)
    y_train_full = train_df_final['log_price']
    print(f"X_train_full (来自清理后的数据) 维度: {X_train_full.shape}")
else:
    raise ValueError("'log_price' 列未在最终训练集中找到！")

# b. 分离测试集的 X 和 ID
if 'ID' in test_df_final.columns:
    X_test = test_df_final.drop('ID', axis=1)
    test_ids = test_df_final['ID'] 
    print(f"X_test 维度: {X_test.shape}")
else:
    print("警告: 最终测试集中未找到 ID 列。将生成虚拟 ID。")
    X_test = test_df_final.copy()
    test_ids = pd.Series(range(len(X_test)), name="ID")

# c. 验证列匹配 (上一步已做过，这里再次确认)
if not all(X_train_full.columns == X_test.columns):
     print("警告: 训练集和测试集列不匹配，尝试重新对齐...")
     # (省略对齐代码 - 假设上一阶段已成功)
     try:
        X_test = X_test[X_train_full.columns]
        if not all(X_train_full.columns == X_test.columns):
             raise ValueError("列对齐失败")
        print("重新对齐成功。")
     except Exception as e:
         raise ValueError(f"训练集和测试集的特征列不匹配！请检查处理流程。{e}")


print(f"X_train_full 维度: {X_train_full.shape}")
print(f"X_test 维度: {X_test.shape}")

# --- 2. 创建预处理管道 (StandardScaler) ---
print("\n--- 2. 正在创建预处理管道 (StandardScaler)... ---")
# 因为所有列都已是数值型 (OHE 完成)，我们只需要对所有列进行缩放
pipeline_ols = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

# --- 3. 训练 OLS 模型 (使用 Pipeline) ---
print("\n--- 3. 正在训练 OLS (Linear Regression) 模型... ---")
# 拟合整个管道
pipeline_ols.fit(X_train_full, y_train_full)

# --- 4. 计算指标 (在原始价格水平上) ---

# a. 反向转换真实的 y_train 值
y_train_orig = np.expm1(y_train_full)

# b. In-sample (样本内) MAE
y_pred_train_log = pipeline_ols.predict(X_train_full) 
y_pred_train_orig = np.expm1(y_pred_train_log)
mae_in_sample = mean_absolute_error(y_train_orig, y_pred_train_orig)

# c. Out-of-sample (样本外) MAE - 使用临时的 test split
X_train_temp, X_val_temp, y_train_temp, y_val_temp = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=111
)
# (在 80% 数据上拟合管道)
pipeline_ols.fit(X_train_temp, y_train_temp) 
y_pred_val_log = pipeline_ols.predict(X_val_temp) # 在 20% 数据上预测
mae_out_of_sample = mean_absolute_error(np.expm1(y_val_temp), np.expm1(y_pred_val_log))
del X_train_temp, X_val_temp, y_train_temp, y_val_temp # 清理

# d. 6-fold Cross-validation (6 折交叉验证) MAE
print("--- 正在运行 6-fold Cross-Validation... ---")

# (自定义 MAE 评估器)
def original_price_mae_scorer(y_log, y_pred_log):
    y_orig = np.expm1(y_log)
    y_pred_orig = np.expm1(y_pred_log)
    y_pred_orig = np.nan_to_num(y_pred_orig, nan=0.0, posinf=np.finfo(np.float64).max, neginf=0.0)
    y_pred_orig = np.clip(y_pred_orig, 0, None)
    return mean_absolute_error(y_orig, y_pred_orig)

custom_mae_scorer = make_scorer(original_price_mae_scorer, greater_is_better=False)
kf = KFold(n_splits=6, shuffle=True, random_state=111) 

# (在完整的 X_train_full, y_train_full 上运行 CV)
cv_scores = cross_val_score(
    pipeline_ols, # 传递整个管道
    X_train_full, 
    y_train_full, 
    cv=kf, 
    scoring=custom_mae_scorer,
    n_jobs=-1 # 使用所有 CPU
)
mae_cv = -np.mean(cv_scores)

# --- 5. 报告结果 ---
print("\n--- OLS 模型性能 (MAE in Original Price) ---")
print(f"In-sample MAE:     {mae_in_sample:,.2f}")
print(f"Out-of-sample MAE (local validation): {mae_out_of_sample:,.2f}")
print(f"6-Fold CV MAE:     {mae_cv:,.2f}")

ols_results = {
    'Metrics': 'OLS (新预处理)',
    'In sample': mae_in_sample,
    'out of sample': mae_out_of_sample, 
    'Cross-validation': mae_cv
}

print("\n--- OLS 结果表格 (用于报告) ---")
print(pd.DataFrame([ols_results]).to_markdown(index=False, floatfmt=",.2f"))

# --- 6. 生成预测文件 (prediction_price_ols.csv) ---
print("\n--- 6. 正在生成 prediction_price_ols.csv 文件... ---")

# a. (重要) 在 *完整* 的训练数据上重新拟合管道
print("--- 正在完整训练数据上重新拟合模型... ---")
pipeline_ols.fit(X_train_full, y_train_full) 

# b. 使用拟合好的管道预测 X_test
y_pred_test_log = pipeline_ols.predict(X_test)

# c. 反向转换并处理异常值
y_pred_test_orig = np.expm1(y_pred_test_log)
fallback_price = np.expm1(y_train_full.median()) 
y_pred_test_orig = np.nan_to_num(y_pred_test_orig, nan=fallback_price, posinf=np.finfo(np.float64).max, neginf=0.0) 
y_pred_test_orig = np.clip(y_pred_test_orig, 0, None) 

# d. 创建提交 DataFrame
submission_df_ols = pd.DataFrame({'ID': test_ids, 'Price': y_pred_test_orig})

# e. 保存为 CSV 文件
submission_filename_ols = 'prediction_price_ols.csv'
submission_df_ols.to_csv(submission_filename_ols, index=False)

print(f"预测结果已保存到 '{submission_filename_ols}'")
print("\n--- OLS 模型处理完毕 ---")

--- 1. 准备训练和测试数据 ---
X_train_full (来自清理后的数据) 维度: (101252, 224)
X_test 维度: (34017, 224)
X_train_full 维度: (101252, 224)
X_test 维度: (34017, 224)

--- 2. 正在创建预处理管道 (StandardScaler)... ---

--- 3. 正在训练 OLS (Linear Regression) 模型... ---
--- 正在运行 6-fold Cross-Validation... ---

--- OLS 模型性能 (MAE in Original Price) ---
In-sample MAE:     453,136.21
Out-of-sample MAE (local validation): 451,941.78
6-Fold CV MAE:     454,604.57

--- OLS 结果表格 (用于报告) ---
| Metrics        |   In sample |   out of sample |   Cross-validation |
|:---------------|------------:|----------------:|-------------------:|
| OLS (新预处理) |  453,136.21 |      451,941.78 |         454,604.57 |

--- 6. 正在生成 prediction_price_ols.csv 文件... ---
--- 正在完整训练数据上重新拟合模型... ---
预测结果已保存到 'prediction_price_ols.csv'

--- OLS 模型处理完毕 ---


In [11]:
# --- [ 独立单元 - 运行 Ridge 模型 (带 GridSearchCV) ] ---
# (请在 b9daea9b 单元格运行后，在新的单元格中运行此代码)

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge # 1. 导入 Ridge
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.pipeline import Pipeline
import warnings

warnings.filterwarnings('ignore')

print("--- 1. 准备训练和测试数据 (来自 train_df_final 和 test_df_final) ---")

# --- 2. (来自 OLS 单元) 准备 X 和 y ---
# 假设 train_df_final 和 test_df_final 已在内存中
if 'log_price' in train_df_final.columns:
    X_train_full = train_df_final.drop('log_price', axis=1)
    y_train_full = train_df_final['log_price']
else:
    raise ValueError("'log_price' 列未在最终训练集中找到！")

if 'ID' in test_df_final.columns:
    X_test = test_df_final.drop('ID', axis=1)
    test_ids = test_df_final['ID']
else:
    raise ValueError("最终测试集中未找到 ID 列！")

# (安全检查) 确保列顺序一致
if not all(X_train_full.columns == X_test.columns):
    print("--- 警告: 训练集和测试集列不匹配，正在重新对齐... ---")
    try:
        X_test = X_test[X_train_full.columns]
        print("--- 重新对齐成功 ---")
    except Exception as e:
         raise ValueError(f"训练集和测试集的特征列不匹配！{e}")

print(f"X_train_full 维度: {X_train_full.shape}")
print(f"X_test 维度: {X_test.shape}")


# --- 3. (来自 OLS 单元) 定义自定义 MAE 评估器 ---
def original_price_mae_scorer(y_log, y_pred_log):
    """
    计算原始价格尺度上的 MAE。
    """
    y_orig = np.expm1(y_log)
    y_pred_orig = np.expm1(y_pred_log)
    
    # 处理预测中的极端值或 NaN (以防万一)
    y_pred_orig = np.nan_to_num(y_pred_orig, nan=0.0, posinf=np.finfo(np.float64).max, neginf=0.0)
    y_pred_orig = np.clip(y_pred_orig, 0, None)
    
    return mean_absolute_error(y_orig, y_pred_orig)

# 创建一个 scikit-learn 'scorer' 对象
custom_mae_scorer = make_scorer(original_price_mae_scorer, greater_is_better=False)

# 定义 6 折交叉验证 (KFold)
kf = KFold(n_splits=6, shuffle=True, random_state=111)


# --- 4. 【新】创建 Ridge 管道和 GridSearchCV ---
print("\n--- 4. 正在创建 Ridge 预处理管道... ---")

# 管道保持不变：先缩放，后回归

pipeline_ridge = Pipeline(steps=[
    ('scaler',StandardScaler()), # <<< 2. 将 StandardScaler() 替换为 RobustScaler()
    ('regressor', Ridge())
])
# 定义要搜索的 alpha (正则化强度) 网格
# (alpha 越大，正则化越强)
# 我们使用一个对数间隔的范围来测试
param_grid_ridge = {
    'regressor__alpha': [1046]
}

# --- 5. 【新】设置并运行 GridSearchCV ---
print("\n--- 5. 正在为 Ridge 运行 6-fold GridSearchCV... ---")
print(f"搜索的参数网格: {param_grid_ridge}")

grid_search_ridge = GridSearchCV(
    estimator=pipeline_ridge,    # 我们的管道
    param_grid=param_grid_ridge, # 要搜索的 alpha
    scoring=custom_mae_scorer,   # 使用自定义的 MAE 评估器
    cv=kf,                       # 6-fold
    n_jobs=-1,                   # 使用所有 CPU 核心
    verbose=2                    # 显示详细进度
)

# (***) 开始训练 (***)
grid_search_ridge.fit(X_train_full, y_train_full)

# --- 6. 【新】获取最佳模型和分数 ---
print("\n--- GridSearchCV 训练完成 ---")
best_ridge_model = grid_search_ridge.best_estimator_
best_mae_cv_ridge = -grid_search_ridge.best_score_  # (Scorer 返回负值, 取反)
best_alpha = grid_search_ridge.best_params_['regressor__alpha']

print(f"最佳 Alpha (正则化强度): {best_alpha}")


# --- 7. (来自 OLS 单元) 计算 In-sample 和 Out-of-sample MAE ---
# (为了完成作业报告)

# a. In-sample (样本内) MAE
# (使用在 GCV 中找到的最佳模型)
y_pred_train_log = best_ridge_model.predict(X_train_full)
y_train_orig = np.expm1(y_train_full)
y_pred_train_orig = np.expm1(y_pred_train_log)
mae_in_sample = mean_absolute_error(y_train_orig, y_pred_train_orig)


# b. Out-of-sample (样本外) MAE - 使用 80/20 划分
# (作业要求)
X_train_temp, X_val_temp, y_train_temp, y_val_temp = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=111
)

# (我们必须在 80% 的数据上重新拟合最佳模型)
# (GridSearchCV 内部的模型是在 k-fold 上训练的,
#  而 best_ridge_model 是在 *全部* X_train_full 上 refit 的)

# (创建一个使用最佳 alpha 的新管道)
pipeline_ridge_best = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('regressor', Ridge(alpha=best_alpha)) 
])
# (仅在 80% 的数据上拟合)
pipeline_ridge_best.fit(X_train_temp, y_train_temp) 

# (在 20% 的数据上预测)
y_pred_val_log = pipeline_ridge_best.predict(X_val_temp) 
mae_out_of_sample = mean_absolute_error(np.expm1(y_val_temp), np.expm1(y_pred_val_log))


# --- 8. 报告结果 ---
print("\n--- Ridge 模型性能 (MAE in Original Price) ---")
print(f"最佳 Alpha:        {best_alpha}")
print(f"In-sample MAE:     {mae_in_sample:,.2f}")
print(f"Out-of-sample MAE (80/20 split): {mae_out_of_sample:,.2f}")
print(f"6-Fold CV MAE:     {best_mae_cv_ridge:,.2f}")

# (生成作业要求的表格)
ridge_results = {
    'Metrics': 'Ridge (新预处理)',
    'In sample': mae_in_sample,
    'out of sample': mae_out_of_sample, 
    'Cross-validation': best_mae_cv_ridge
}
print("\n--- Ridge 结果表格 (用于报告) ---")
print(pd.DataFrame([ridge_results]).to_markdown(index=False, floatfmt=",.2f"))


# --- 9. 生成预测文件 (prediction_price_ridge.csv) ---
print("\n--- 9. 正在生成 prediction_price_ridge.csv 文件... ---")

# (GridSearchCV 对象在 .fit() 之后，会自动在 *全部* 训练数据上
#  重新训练一个最佳模型，即 grid_search_ridge.best_estimator_)
# (我们可以直接使用它来预测 X_test)
y_pred_test_log = grid_search_ridge.predict(X_test)

# c. 反向转换并处理异常值
y_pred_test_orig = np.expm1(y_pred_test_log)
fallback_price = np.expm1(y_train_full.median()) 
y_pred_test_orig = np.nan_to_num(y_pred_test_orig, nan=fallback_price, posinf=np.finfo(np.float64).max, neginf=0.0) 
y_pred_test_orig = np.clip(y_pred_test_orig, 0, None) 

# d. 创建提交 DataFrame
submission_df_ridge = pd.DataFrame({'ID': test_ids, 'Price': y_pred_test_orig})

# e. 保存为 CSV 文件
submission_filename_ridge = 'prediction_price_ridge.csv'
submission_df_ridge.to_csv(submission_filename_ridge, index=False)

print(f"预测结果已保存到 '{submission_filename_ridge}'")
print("\n--- Ridge 模型处理完毕 ---")

--- 1. 准备训练和测试数据 (来自 train_df_final 和 test_df_final) ---
X_train_full 维度: (101252, 224)
X_test 维度: (34017, 224)

--- 4. 正在创建 Ridge 预处理管道... ---

--- 5. 正在为 Ridge 运行 6-fold GridSearchCV... ---
搜索的参数网格: {'regressor__alpha': [1046]}
Fitting 6 folds for each of 1 candidates, totalling 6 fits

--- GridSearchCV 训练完成 ---
最佳 Alpha (正则化强度): 1046

--- Ridge 模型性能 (MAE in Original Price) ---
最佳 Alpha:        1046
In-sample MAE:     453,952.22
Out-of-sample MAE (80/20 split): 452,020.99
6-Fold CV MAE:     455,186.89

--- Ridge 结果表格 (用于报告) ---
| Metrics          |   In sample |   out of sample |   Cross-validation |
|:-----------------|------------:|----------------:|-------------------:|
| Ridge (新预处理) |  453,952.22 |      452,020.99 |         455,186.89 |

--- 9. 正在生成 prediction_price_ridge.csv 文件... ---
预测结果已保存到 'prediction_price_ridge.csv'

--- Ridge 模型处理完毕 ---


In [12]:
# --- [ 独立单元 - Ridge 模型残差分析 ] ---
# (请在你运行 Ridge 模型的代码之后，在新的单元格中运行此代码)
# 假设以下变量已在之前的单元格中计算并可用:
# y_train_full: 真实的 log_price (训练集)
# y_pred_train_log: 使用 best_ridge_model 在 X_train_full 上的预测 log_price
# X_train_full: 最终的训练特征 DataFrame (用于对照分析)

import pandas as pd
import numpy as np

print("\n--- 10. 正在进行 Ridge 模型残差分析 (基于训练集)... ---")

# --- a. 计算残差 (对数尺度) ---
if 'y_train_full' in locals() and 'y_pred_train_log' in locals():
    residuals_log = y_train_full - y_pred_train_log
    print(f"已计算残差，共 {len(residuals_log)} 个。")
else:
    raise NameError("需要先运行 Ridge 模型训练单元以获得 y_train_full 和 y_pred_train_log")

# --- b. 残差基本统计信息 ---
print("\n--- 残差基本统计信息 (对数尺度) ---")
# 使用 Series 的 describe() 方法获取统计量
residual_stats = pd.Series(residuals_log).describe()
print(residual_stats.to_string())
# 检查：均值是否接近 0？标准差多大？最大/最小值表明是否有极端偏差？

# --- c. 残差 vs. 预测值 分析 (检查非线性和异方差性) ---
print("\n--- 残差 vs. 预测值 分析 (按预测值分位数) ---")
try:
    # 将预测值分为 N 个分位数区间 (例如 5 个)
    num_quantiles = 5
    predicted_quantiles = pd.qcut(y_pred_train_log, q=num_quantiles, labels=False, duplicates='drop')

    # 按分位数区间分组计算残差的均值和标准差
    grouped_residuals = pd.DataFrame({'residual': residuals_log, 'quantile': predicted_quantiles})
    quantile_analysis = grouped_residuals.groupby('quantile')['residual'].agg(['mean', 'std', 'count'])

    print(f"按 {num_quantiles} 个预测值分位数区间统计的残差:")
    print(quantile_analysis.to_string(float_format="%.4f"))
    # 检查：
    # 1. 各区间 'mean' 是否都接近 0？如果系统性偏离0（例如随区间递增/减），可能存在未捕捉的非线性关系。
    # 2. 各区间 'std' 是否大致相等？如果标准差随区间系统性变化（例如变大或变小），可能存在异方差性。

except ValueError as e:
    print(f"无法按预测值分位数分析残差（可能是预测值分布问题）: {e}")
except NameError:
     print("无法按预测值分位数分析残差，请确保 y_pred_train_log 已定义。")


# --- d. 识别极端残差 (潜在异常值) ---
print("\n--- 极端残差识别 ---")
n_extreme = 10 # 显示最大/最小的 N 个残差
largest_positive_residuals = pd.Series(residuals_log).nlargest(n_extreme)
largest_negative_residuals = pd.Series(residuals_log).nsmallest(n_extreme)

print(f"最大的 {n_extreme} 个正残差 (预测值远低于实际值):")
print(largest_positive_residuals.to_string(float_format="%.4f"))
print(f"\n最大的 {n_extreme} 个负残差 (预测值远高于实际值):")
print(largest_negative_residuals.to_string(float_format="%.4f"))
# 检查：这些极端残差对应的样本点是否有特殊之处？（需要对照原始数据或特征值分析）

# --- e. 残差 vs. 关键特征 分析 (检查特定特征的拟合情况) ---
print("\n--- 残差 vs. 关键特征 分析 (按特征分位数) ---")
key_features_for_residual_analysis = ['log_建筑面积', '房龄'] # 选择你关心的几个核心特征

for feature in key_features_for_residual_analysis:
    if feature in X_train_full.columns:
        print(f"\n--- 分析残差 vs. '{feature}' ---")
        try:
            # 将特征值分为 N 个分位数区间
            feature_quantiles = pd.qcut(X_train_full[feature], q=num_quantiles, labels=False, duplicates='drop')

            # 按特征分位数区间分组计算残差均值
            grouped_residuals_feature = pd.DataFrame({'residual': residuals_log, 'quantile': feature_quantiles})
            feature_quantile_analysis = grouped_residuals_feature.groupby('quantile')['residual'].agg(['mean', 'count'])

            print(f"按 '{feature}' 的 {num_quantiles} 个分位数区间统计的残差均值:")
            print(feature_quantile_analysis.to_string(float_format="%.4f"))
            # 检查：各区间 'mean' 是否都接近 0？如果残差均值随特征值系统性变化，说明模型对该特征的拟合可能仍有改进空间（例如，非线性关系未完全捕捉）。

        except ValueError as e:
            print(f"无法按 '{feature}' 的分位数分析残差（可能是特征值分布问题）: {e}")
        except Exception as e_gen:
             print(f"分析残差 vs. '{feature}' 时出错: {e_gen}")
    else:
        print(f"特征 '{feature}' 不在 X_train_full 中，跳过分析。")

print("\n--- 残差分析完毕 ---")


--- 10. 正在进行 Ridge 模型残差分析 (基于训练集)... ---
已计算残差，共 101252 个。

--- 残差基本统计信息 (对数尺度) ---
count    1.012520e+05
mean    -2.039375e-15
std      2.543364e-01
min     -2.087020e+00
25%     -1.523277e-01
50%      3.170698e-03
75%      1.503200e-01
max      1.678598e+00

--- 残差 vs. 预测值 分析 (按预测值分位数) ---
按 5 个预测值分位数区间统计的残差:
            mean    std  count
quantile                      
0        -0.0008 0.2356  20251
1        -0.0125 0.2308  20250
2        -0.0102 0.2493  20250
3         0.0085 0.2677  20250
4         0.0151 0.2834  20251

--- 极端残差识别 ---
最大的 10 个正残差 (预测值远低于实际值):
78923   1.6786
55518   1.5659
56969   1.5558
63247   1.3040
5974    1.2308
63767   1.1997
69643   1.1968
89960   1.1834
81229   1.1821
70051   1.1757

最大的 10 个负残差 (预测值远高于实际值):
6327    -2.0870
88214   -1.6943
89931   -1.3269
93763   -1.2989
96268   -1.2597
71079   -1.2432
69119   -1.2021
68895   -1.1934
68211   -1.1402
27010   -1.1360

--- 残差 vs. 关键特征 分析 (按特征分位数) ---

--- 分析残差 vs. 'log_建筑面积' ---
按 'log_建筑面积' 的 5 个分位数区间统计的残差均值

In [13]:
# --- [ 独立单元 - 运行 Lasso 模型 (带 GridSearchCV) ] ---
# (请在 b9daea9b 单元格运行后，在新的单元格中运行此代码)

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso # 1. 导入 Lasso
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.pipeline import Pipeline
import warnings

warnings.filterwarnings('ignore')

print("--- 1. 准备训练和测试数据 (来自 train_df_final 和 test_df_final) ---")

# --- 2. (来自 OLS 单元) 准备 X 和 y ---
# 假设 train_df_final 和 test_df_final 已在内存中
if 'log_price' in train_df_final.columns:
    X_train_full = train_df_final.drop('log_price', axis=1)
    y_train_full = train_df_final['log_price']
else:
    raise ValueError("'log_price' 列未在最终训练集中找到！")

if 'ID' in test_df_final.columns:
    X_test = test_df_final.drop('ID', axis=1)
    test_ids = test_df_final['ID']
else:
    raise ValueError("最终测试集中未找到 ID 列！")

# (安全检查) 确保列顺序一致
if not all(X_train_full.columns == X_test.columns):
    print("--- 警告: 训练集和测试集列不匹配，正在重新对齐... ---")
    try:
        X_test = X_test[X_train_full.columns]
        print("--- 重新对齐成功 ---")
    except Exception as e:
         raise ValueError(f"训练集和测试集的特征列不匹配！{e}")

print(f"X_train_full 维度: {X_train_full.shape}")
print(f"X_test 维度: {X_test.shape}")


# --- 3. (来自 OLS 单元) 定义自定义 MAE 评估器 ---
def original_price_mae_scorer(y_log, y_pred_log):
    """
    计算原始价格尺度上的 MAE。
    """
    y_orig = np.expm1(y_log)
    y_pred_orig = np.expm1(y_pred_log)
    
    # 处理预测中的极端值或 NaN (以防万一)
    y_pred_orig = np.nan_to_num(y_pred_orig, nan=0.0, posinf=np.finfo(np.float64).max, neginf=0.0)
    y_pred_orig = np.clip(y_pred_orig, 0, None)
    
    return mean_absolute_error(y_orig, y_pred_orig)

# 创建一个 scikit-learn 'scorer' 对象
custom_mae_scorer = make_scorer(original_price_mae_scorer, greater_is_better=False)

# 定义 6 折交叉验证 (KFold)
kf = KFold(n_splits=6, shuffle=True, random_state=111)


# --- 4. 【新】创建 Lasso 管道和 GridSearchCV ---
print("\n--- 4. 正在创建 Lasso 预处理管道... ---")

# 管道保持不变：先缩放，后回归
pipeline_lasso = Pipeline(steps=[
    ('scaler', StandardScaler()),
    # 2. 使用 Lasso() 并增加 max_iter
    ('regressor', Lasso(max_iter=3000, random_state=111)) 
])

# 定义要搜索的 alpha (正则化强度) 网格
# (Lasso 的 alpha 通常需要设置得非常小)
param_grid_lasso = {
    'regressor__alpha': [0.001]
}

# --- 5. 【新】设置并运行 GridSearchCV ---
print("\n--- 5. 正在为 Lasso 运行 6-fold GridSearchCV... ---")
print(f"搜索的参数网格: {param_grid_lasso}")

grid_search_lasso = GridSearchCV(
    estimator=pipeline_lasso,    # 我们的管道
    param_grid=param_grid_lasso, # 要搜索的 alpha
    scoring=custom_mae_scorer,   # 使用自定义的 MAE 评估器
    cv=kf,                       # 6-fold
    n_jobs=-1,                   # 使用所有 CPU 核心
    verbose=2                    # 显示详细进度
)

# (***) 开始训练 (***)
grid_search_lasso.fit(X_train_full, y_train_full)

# --- 6. 【新】获取最佳模型和分数 ---
print("\n--- GridSearchCV 训练完成 ---")
best_lasso_model = grid_search_lasso.best_estimator_
best_mae_cv_lasso = -grid_search_lasso.best_score_  # (Scorer 返回负值, 取反)
best_alpha = grid_search_lasso.best_params_['regressor__alpha']

print(f"最佳 Alpha (正则化强度): {best_alpha}")

# (【新】Lasso 独有) 检查有多少特征被保留
try:
    # 提取 Lasso 回归器
    lasso_regressor = best_lasso_model.named_steps['regressor']
    # 获取系数
    coefficients = lasso_regressor.coef_
    # 计算非零系数的数量
    n_features_selected = np.sum(coefficients != 0)
    total_features = len(coefficients)
    print(f"Lasso 特征选择: 保留了 {n_features_selected} / {total_features} 个特征")
except Exception as e:
    print(f"无法获取Lasso系数: {e}")


# --- 7. (来自 OLS 单元) 计算 In-sample 和 Out-of-sample MAE ---
# (为了完成作业报告)

# a. In-sample (样本内) MAE
y_pred_train_log = best_lasso_model.predict(X_train_full)
y_train_orig = np.expm1(y_train_full)
y_pred_train_orig = np.expm1(y_pred_train_log)
mae_in_sample = mean_absolute_error(y_train_orig, y_pred_train_orig)


# b. Out-of-sample (样本外) MAE - 使用 80/20 划分
X_train_temp, X_val_temp, y_train_temp, y_val_temp = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=111
)

# (创建一个使用最佳 alpha 的新管道)
pipeline_lasso_best = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('regressor', Lasso(alpha=best_alpha, max_iter=3000, random_state=111)) 
])
# (仅在 80% 的数据上拟合)
pipeline_lasso_best.fit(X_train_temp, y_train_temp) 

# (在 20% 的数据上预测)
y_pred_val_log = pipeline_lasso_best.predict(X_val_temp) 
mae_out_of_sample = mean_absolute_error(np.expm1(y_val_temp), np.expm1(y_pred_val_log))


# --- 8. 报告结果 ---
print("\n--- Lasso 模型性能 (MAE in Original Price) ---")
print(f"最佳 Alpha:        {best_alpha}")
print(f"In-sample MAE:     {mae_in_sample:,.2f}")
print(f"Out-of-sample MAE (80/20 split): {mae_out_of_sample:,.2f}")
print(f"6-Fold CV MAE:     {best_mae_cv_lasso:,.2f}")
print(f"Lasso 保留特征: {n_features_selected} / {total_features}")

# (生成作业要求的表格)
lasso_results = {
    'Metrics': 'Lasso (新预处理)',
    'In sample': mae_in_sample,
    'out of sample': mae_out_of_sample, 
    'Cross-validation': best_mae_cv_lasso
}
print("\n--- Lasso 结果表格 (用于报告) ---")
print(pd.DataFrame([lasso_results]).to_markdown(index=False, floatfmt=",.2f"))


# --- 9. 生成预测文件 (prediction_price_lasso.csv) ---
print("\n--- 9. 正在生成 prediction_price_lasso.csv 文件... ---")

# (使用 GCV 找到的最佳模型预测 X_test)
y_pred_test_log = grid_search_lasso.predict(X_test)

# c. 反向转换并处理异常值
y_pred_test_orig = np.expm1(y_pred_test_log)
fallback_price = np.expm1(y_train_full.median()) 
y_pred_test_orig = np.nan_to_num(y_pred_test_orig, nan=fallback_price, posinf=np.finfo(np.float64).max, neginf=0.0) 
y_pred_test_orig = np.clip(y_pred_test_orig, 0, None) 

# d. 创建提交 DataFrame
submission_df_lasso = pd.DataFrame({'ID': test_ids, 'Price': y_pred_test_orig})

# e. 保存为 CSV 文件
submission_filename_lasso = 'prediction_price_lasso.csv'
submission_df_lasso.to_csv(submission_filename_lasso, index=False)

print(f"预测结果已保存到 '{submission_filename_lasso}'")
print("\n--- Lasso 模型处理完毕 ---")

--- 1. 准备训练和测试数据 (来自 train_df_final 和 test_df_final) ---
X_train_full 维度: (101252, 224)
X_test 维度: (34017, 224)

--- 4. 正在创建 Lasso 预处理管道... ---

--- 5. 正在为 Lasso 运行 6-fold GridSearchCV... ---
搜索的参数网格: {'regressor__alpha': [0.001]}
Fitting 6 folds for each of 1 candidates, totalling 6 fits

--- GridSearchCV 训练完成 ---
最佳 Alpha (正则化强度): 0.001
Lasso 特征选择: 保留了 192 / 224 个特征

--- Lasso 模型性能 (MAE in Original Price) ---
最佳 Alpha:        0.001
In-sample MAE:     454,916.36
Out-of-sample MAE (80/20 split): 452,899.78
6-Fold CV MAE:     456,015.32
Lasso 保留特征: 192 / 224

--- Lasso 结果表格 (用于报告) ---
| Metrics          |   In sample |   out of sample |   Cross-validation |
|:-----------------|------------:|----------------:|-------------------:|
| Lasso (新预处理) |  454,916.36 |      452,899.78 |         456,015.32 |

--- 9. 正在生成 prediction_price_lasso.csv 文件... ---
预测结果已保存到 'prediction_price_lasso.csv'

--- Lasso 模型处理完毕 ---


In [37]:
# --- [ 独立单元 - 运行 随机森林 (Random Forest) 模型 (带 RandomizedSearchCV) ] ---
# (请在 STAGE 8 清理完成，获得 train_df_final 和 test_df_final 后，在新的单元格中运行此代码)

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler # 可以选择你最终使用的 Scaler
from sklearn.ensemble import RandomForestRegressor # 1. 导入 RandomForestRegressor
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.pipeline import Pipeline
import warnings
import time # 用于记录时间

warnings.filterwarnings('ignore')

print("--- 1. 准备训练和测试数据 (来自 train_df_final 和 test_df_final) ---")

# --- 2. 准备 X 和 y ---
# 假设 train_df_final 和 test_df_final 已在内存中
if 'log_price' in train_df_final.columns:
    X_train_full = train_df_final.drop('log_price', axis=1)
    y_train_full = train_df_final['log_price']
else:
    raise ValueError("'log_price' 列未在最终训练集中找到！")

if 'ID' in test_df_final.columns:
    X_test = test_df_final.drop('ID', axis=1)
    test_ids = test_df_final['ID']
else:
    raise ValueError("最终测试集中未找到 ID 列！")

# (安全检查) 确保列顺序一致
if not all(X_train_full.columns == X_test.columns):
    print("--- 警告: 训练集和测试集列不匹配，正在重新对齐... ---")
    try:
        X_test = X_test[X_train_full.columns]
        print("--- 重新对齐成功 ---")
    except Exception as e:
         raise ValueError(f"训练集和测试集的特征列不匹配！{e}")

print(f"X_train_full 维度: {X_train_full.shape}")
print(f"X_test 维度: {X_test.shape}")


# --- 3. 定义自定义 MAE 评估器 ---
# (这段代码在你之前的单元格中应该已经定义过了，这里为了独立性再次包含)
def original_price_mae_scorer(y_log, y_pred_log):
    """计算原始价格尺度上的 MAE。"""
    y_orig = np.expm1(y_log)
    y_pred_orig = np.expm1(y_pred_log)
    y_pred_orig = np.nan_to_num(y_pred_orig, nan=0.0, posinf=np.finfo(np.float64).max, neginf=0.0)
    y_pred_orig = np.clip(y_pred_orig, 0, None)
    return mean_absolute_error(y_orig, y_pred_orig)

custom_mae_scorer = make_scorer(original_price_mae_scorer, greater_is_better=False)

# 定义 6 折交叉验证 (KFold)
kf = KFold(n_splits=6, shuffle=True, random_state=111)


# --- 4. 【新】创建 随机森林 管道和 RandomizedSearchCV ---
print("\n--- 4. 正在创建 随机森林 预处理管道... ---")

# 选择 Scaler (根据你之前的选择，StandardScaler 或 RobustScaler)
# scaler_to_use = StandardScaler()
scaler_to_use = RobustScaler() # 如果你决定使用 RobustScaler

pipeline_rf = Pipeline(steps=[
    ('scaler', scaler_to_use),
    # 2. 使用 RandomForestRegressor()
    # n_jobs=-1 使用所有 CPU 核心加速训练
    # random_state=111 保证结果可复现
    ('regressor', RandomForestRegressor(n_jobs=-1, random_state=111))
])

# --- 5. 【新】定义随机森林的超参数搜索空间 ---
# RandomizedSearchCV 比 GridSearchCV 更快地探索大范围参数
param_dist_rf = {
    'regressor__n_estimators': [100, 200, 300],       # 树的数量
    'regressor__max_depth': [10, 20, 30, None],       # 树的最大深度 (None 表示不限制)
    'regressor__min_samples_split': [2, 5, 10],     # 节点分裂所需的最小样本数
    'regressor__min_samples_leaf': [1, 3, 5],        # 叶节点所需的最小样本数
    'regressor__max_features': ['sqrt', 'log2', 1.0] # 每次分裂考虑的特征比例/数量 ('sqrt', 'log2', 或 1.0 代表全部)
}
# 注意: 这个搜索空间可以根据你的计算资源和时间进行调整

n_iter_search = 10 # 随机搜索的迭代次数 (尝试的参数组合数量)，可以适当增加以获得更好结果，但会增加时间

# --- 6. 【新】设置并运行 RandomizedSearchCV ---
print("\n--- 6. 正在为 随机森林 运行 6-fold RandomizedSearchCV... ---")
print(f"将随机尝试 {n_iter_search} 种参数组合")

random_search_rf = RandomizedSearchCV(
    estimator=pipeline_rf,        # 我们的管道
    param_distributions=param_dist_rf, # 要搜索的参数分布
    n_iter=n_iter_search,         # 随机搜索次数
    scoring=custom_mae_scorer,    # 使用自定义的 MAE 评估器
    cv=kf,                        # 6-fold
    n_jobs=-1,                    # 使用所有 CPU 核心 (GridSearchCV 本身也并行)
    random_state=111,             # 保证随机搜索过程可复现
    verbose=2                     # 显示详细进度
)

# (***) 开始训练 (***)
start_time = time.time()
random_search_rf.fit(X_train_full, y_train_full)
end_time = time.time()
print(f"\n--- RandomizedSearchCV 训练完成 --- (耗时: {end_time - start_time:.2f} 秒)")

# --- 7. 【新】获取最佳模型和分数 ---
best_rf_model = random_search_rf.best_estimator_
best_mae_cv_rf = -random_search_rf.best_score_  # (Scorer 返回负值, 取反)
best_params_rf = random_search_rf.best_params_

print(f"\n最佳参数组合: {best_params_rf}")


# --- 8. 计算 In-sample 和 Out-of-sample MAE ---
# (为了与其他模型比较)

# a. In-sample (样本内) MAE
y_pred_train_log_rf = best_rf_model.predict(X_train_full)
y_train_orig = np.expm1(y_train_full) # 原始 y_train
y_pred_train_orig_rf = np.expm1(y_pred_train_log_rf)
mae_in_sample_rf = mean_absolute_error(y_train_orig, y_pred_train_orig_rf)

# b. Out-of-sample (样本外) MAE - 使用 80/20 划分
# (注意：这里的 OOS MAE 是基于一次 80/20 划分的估计，CV MAE 通常更可靠)
X_train_temp_rf, X_val_temp_rf, y_train_temp_rf, y_val_temp_rf = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=111
)
# 需要用找到的最佳参数重新拟合模型 (因为 best_rf_model 是在全数据上 refit 的)
# 这里直接使用 best_rf_model 在验证集上预测可能稍微高估性能，但更方便
y_pred_val_log_rf = best_rf_model.predict(X_val_temp_rf) # 使用已在全数据训练的最佳模型
mae_out_of_sample_rf = mean_absolute_error(np.expm1(y_val_temp_rf), np.expm1(y_pred_val_log_rf))


# --- 9. 报告结果 ---
print("\n--- 随机森林 模型性能 (MAE in Original Price) ---")
print(f"最佳参数: {best_params_rf}")
print(f"In-sample MAE:     {mae_in_sample_rf:,.2f}")
print(f"Out-of-sample MAE (80/20 split): {mae_out_of_sample_rf:,.2f}")
print(f"6-Fold CV MAE (来自搜索): {best_mae_cv_rf:,.2f}")

# (生成结果表格)
rf_results = {
    'Metrics': 'Random Forest',
    'In sample': mae_in_sample_rf,
    'out of sample': mae_out_of_sample_rf,
    'Cross-validation': best_mae_cv_rf
}
print("\n--- 随机森林 结果表格 (用于报告) ---\n")
# 将新结果添加到之前的结果 DataFrame (如果存在的话) 或单独打印
# 假设 all_results_df 是之前包含 OLS, Ridge 等结果的 DataFrame
# all_results_df = pd.concat([all_results_df, pd.DataFrame([rf_results])], ignore_index=True)
# print(all_results_df.to_markdown(index=False, floatfmt=",.2f"))
# 或者单独打印
print(pd.DataFrame([rf_results]).to_markdown(index=False, floatfmt=",.2f"))


# --- 10. 生成预测文件 (prediction_price_rf.csv) ---
print("\n--- 10. 正在生成 prediction_price_rf.csv 文件... ---")

# (RandomizedSearchCV 对象在 .fit() 之后，会自动在 *全部* 训练数据上
#  用最佳参数重新训练一个模型，存储在 best_estimator_ 中)
# (我们可以直接使用它来预测 X_test)
y_pred_test_log_rf = best_rf_model.predict(X_test)

# c. 反向转换并处理异常值
y_pred_test_orig_rf = np.expm1(y_pred_test_log_rf)
fallback_price_rf = np.expm1(y_train_full.median()) # 仍然使用中位数作为备用
y_pred_test_orig_rf = np.nan_to_num(y_pred_test_orig_rf, nan=fallback_price_rf, posinf=np.finfo(np.float64).max, neginf=0.0)
y_pred_test_orig_rf = np.clip(y_pred_test_orig_rf, 0, None)

# d. 创建提交 DataFrame
submission_df_rf = pd.DataFrame({'ID': test_ids, 'Price': y_pred_test_orig_rf})

# e. 保存为 CSV 文件
submission_filename_rf = 'prediction_price_rf.csv'
submission_df_rf.to_csv(submission_filename_rf, index=False)

print(f"预测结果已保存到 '{submission_filename_rf}'")
print("\n--- 随机森林 模型处理完毕 ---")

--- 1. 准备训练和测试数据 (来自 train_df_final 和 test_df_final) ---
X_train_full 维度: (101252, 224)
X_test 维度: (34017, 224)

--- 4. 正在创建 随机森林 预处理管道... ---

--- 6. 正在为 随机森林 运行 6-fold RandomizedSearchCV... ---
将随机尝试 10 种参数组合
Fitting 6 folds for each of 10 candidates, totalling 60 fits

--- RandomizedSearchCV 训练完成 --- (耗时: 1010.22 秒)

最佳参数组合: {'regressor__n_estimators': 300, 'regressor__min_samples_split': 5, 'regressor__min_samples_leaf': 5, 'regressor__max_features': 1.0, 'regressor__max_depth': 30}

--- 随机森林 模型性能 (MAE in Original Price) ---
最佳参数: {'regressor__n_estimators': 300, 'regressor__min_samples_split': 5, 'regressor__min_samples_leaf': 5, 'regressor__max_features': 1.0, 'regressor__max_depth': 30}
In-sample MAE:     145,877.78
Out-of-sample MAE (80/20 split): 141,760.66
6-Fold CV MAE (来自搜索): 225,818.19

--- 随机森林 结果表格 (用于报告) ---

| Metrics       |   In sample |   out of sample |   Cross-validation |
|:--------------|------------:|----------------:|-------------------:|
| Random Forest |  14

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge # 使用 Ridge
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import warnings

warnings.filterwarnings('ignore')

# !!! 警告: TargetEncoder 需要 category_encoders 库
# 在 Jupyter 中，您可能需要先运行: !pip install category_encoders
try:
    from category_encoders import TargetEncoder
except ImportError:
    print("错误: TargetEncoder 未找到。")
    print("请先在您的环境中运行: pip install category_encoders")
    TargetEncoder = None # Define as None if not found

# --- (假设 train_df_final 和 test_df_final 是 'rent' 数据处理后的 DataFrame) ---
# train_df_final: 包含 log_price, 不含 ID
# test_df_final: 包含 ID, 不含 log_price

print("--- 1. 准备训练和测试数据 ('rent' data) ---")

# a. 分离训练集的 X 和 y
if 'log_price' in train_df_final.columns:
    X_train_full = train_df_final.drop('log_price', axis=1)
    y_train_full = train_df_final['log_price']
else:
    raise ValueError("'log_price' 列未在最终训练集中找到！")

# b. 分离测试集的 X 和 ID
if 'ID' in test_df_final.columns:
    X_test = test_df_final.drop('ID', axis=1)
    test_ids = test_df_final['ID']
else:
    print("警告: 最终测试集中未找到 ID 列。将生成虚拟 ID。")
    X_test = test_df_final.copy()
    test_ids = pd.Series(range(len(X_test)), name="ID")

# c. 验证列匹配 (保险起见)
if not all(X_train_full.columns == X_test.columns):
    print("警告: 训练集和测试集列不匹配，尝试重新对齐...")
    # (省略对齐代码 - 假设上一步已完成)
    train_cols_set = set(X_train_full.columns)
    test_cols_set = set(X_test.columns)
    if train_cols_set != test_cols_set:
         raise ValueError("训练集和测试集的特征列不匹配！请检查处理流程。")
    else:
         X_test = X_test[X_train_full.columns]
         print("列已对齐。")

# d. 转换 TargetEncoder 列为 category 类型
target_encode_cols = ['区县', '板块'] # Rent 数据需要 Target Encoding 的列
target_encode_cols = [col for col in target_encode_cols if col in X_train_full.columns]
if target_encode_cols and TargetEncoder is None:
     raise ImportError("TargetEncoder is required but not found.")
if target_encode_cols:
    print(f"--- 正在转换 {target_encode_cols} 为 'category' 类型 ---")
    for col in target_encode_cols:
        X_train_full[col] = X_train_full[col].astype('category')
        X_test[col] = X_test[col].astype('category')

print(f"X_train_full 维度: {X_train_full.shape}")
print(f"X_test 维度: {X_test.shape}")

# --- 2. 创建预处理管道 (与 OLS/Lasso 脚本相同) ---
print("\n--- 2. 正在创建预处理管道... ---")
numeric_cols = [col for col in X_train_full.columns if col not in target_encode_cols]
transformers = []
if target_encode_cols:
    transformers.append(('target_encoder', TargetEncoder(), target_encode_cols))
transformers.append(('standard_scaler', StandardScaler(), numeric_cols))
preprocessor = ColumnTransformer(transformers=transformers, remainder='passthrough')

# --- 3. 定义 Ridge 模型和超参数网格 ---
# (创建 Pipeline，Regressor 更改为 Ridge)
pipeline_ridge = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge(random_state=111)) # 使用 Ridge
])

# 定义要搜索的 alpha (L2 正则化强度) 值
param_grid_ridge = {
    'regressor__alpha': [1000] # 例如: 0.001, 0.01, 0.1, 1, 10, 100, 1000
}

# --- 4. 使用 GridSearchCV 寻找最佳 Alpha ---
print("\n--- 4. 正在使用 GridSearchCV 寻找最佳 Ridge Alpha... ---")

# 定义 MAE 评分器 (与之前相同)
def original_price_mae_scorer(y_log, y_pred_log):
    y_orig = np.expm1(y_log)
    y_pred_orig = np.expm1(y_pred_log)
    y_pred_orig = np.nan_to_num(y_pred_orig, nan=0.0, posinf=np.finfo(np.float64).max, neginf=0.0)
    y_pred_orig = np.clip(y_pred_orig, 0, None)
    return mean_absolute_error(y_orig, y_pred_orig)

neg_mae_scorer = make_scorer(original_price_mae_scorer, greater_is_better=False)

# 使用 6 折交叉验证
kf = KFold(n_splits=6, shuffle=True, random_state=111)

grid_search_ridge = GridSearchCV(
    pipeline_ridge,
    param_grid_ridge,
    cv=kf,
    scoring=neg_mae_scorer,
    n_jobs=-1,
    verbose=1 # 显示进度
)

# 在完整的训练集上运行 GridSearch
grid_search_ridge.fit(X_train_full, y_train_full)

# 获取最佳模型和最佳 alpha
best_model_ridge = grid_search_ridge.best_estimator_
best_alpha_ridge = grid_search_ridge.best_params_['regressor__alpha']

print(f"--- GridSearchCV 完成 ---")
print(f"最佳 Ridge Alpha: {best_alpha_ridge}")
print(f"对应的最佳 CV MAE (原始价格): {-grid_search_ridge.best_score_:,.2f}")

# --- 5. 计算指标 (使用最佳模型) ---

# a. In-sample (样本内) MAE
y_train_orig = np.expm1(y_train_full) # 真实的原始价格
y_pred_train_log = best_model_ridge.predict(X_train_full)
y_pred_train_orig = np.expm1(y_pred_train_log)
mae_in_sample = mean_absolute_error(y_train_orig, y_pred_train_orig)

# b. Out-of-sample (样本外) MAE - 使用临时的 test split
X_train_temp, X_val_temp, y_train_temp, y_val_temp = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=111
)
best_model_ridge.fit(X_train_temp, y_train_temp) # 用最佳 alpha 重新训练 (以正确fit TargetEncoder)
y_pred_val_log = best_model_ridge.predict(X_val_temp)
mae_out_of_sample = mean_absolute_error(np.expm1(y_val_temp), np.expm1(y_pred_val_log))
del X_train_temp, X_val_temp, y_train_temp, y_val_temp # 清理

# c. 6-fold Cross-validation MAE (GridSearch 已经计算过了)
mae_cv = -grid_search_ridge.best_score_

# --- 6. 报告结果 ---
print("\n--- Ridge 模型性能 ('rent' data, MAE in Original Price) ---")
print(f"Best Alpha:        {best_alpha_ridge}")
print(f"In-sample MAE:     {mae_in_sample:,.2f}")
print(f"Out-of-sample MAE (local validation): {mae_out_of_sample:,.2f}")
print(f"6-Fold CV MAE:     {mae_cv:,.2f}")

ridge_results_rent = {
    'Metrics': 'Ridge',
    'In sample': mae_in_sample,
    'out of sample': mae_out_of_sample, # 本地验证集
    'Cross-validation': mae_cv
}

print("\n--- Ridge 结果表格 ('rent' data, 用于报告) ---")
print(pd.DataFrame([ridge_results_rent]).to_markdown(index=False, floatfmt=",.2f"))

# --- 7. 生成预测文件 (prediction_rent_ridge.csv) ---
print("\n--- 7. 正在使用最佳 Alpha 重新训练模型并生成 prediction_rent_ridge.csv 文件... ---")

# a. 使用找到的最佳 Alpha 在 *完整* 训练集上训练最终模型
final_model_ridge = grid_search_ridge.best_estimator_ # GridSearch 默认会 refit
# final_model_ridge.fit(X_train_full, y_train_full) # 可以选择显式 refit

# b. 预测测试集
y_pred_test_log = final_model_ridge.predict(X_test)

# c. 反向转换并处理异常值
y_pred_test_orig = np.expm1(y_pred_test_log)
fallback_price = np.expm1(y_train_full.median()) # 使用训练集的中位数租金作为 fallback
y_pred_test_orig = np.nan_to_num(y_pred_test_orig, nan=fallback_price, posinf=np.finfo(np.float64).max, neginf=0.0)
y_pred_test_orig = np.clip(y_pred_test_orig, 0, None) # 确保租金非负

# d. 创建提交 DataFrame
submission_df_ridge_rent = pd.DataFrame({'ID': test_ids, 'Price': y_pred_test_orig})

# e. 保存为 CSV 文件
submission_filename_ridge_rent = 'prediction_rent_ridge.csv'
submission_df_ridge_rent.to_csv(submission_filename_ridge_rent, index=False)

# Make the submission file available for download
# from google.colab import files # Uncomment if in Colab
# files.download(submission_filename_ridge_rent)

print(f"预测结果已保存到 '{submission_filename_ridge_rent}'") #.") # Comment out if not in Colab
print("\n--- Ridge 模型 ('rent' data) 处理完毕 ---")

--- 1. 准备训练和测试数据 ('rent' data) ---
--- 正在转换 ['区县', '板块'] 为 'category' 类型 ---
X_train_full 维度: (98890, 141)
X_test 维度: (9773, 141)

--- 2. 正在创建预处理管道... ---

--- 4. 正在使用 GridSearchCV 寻找最佳 Ridge Alpha... ---
Fitting 6 folds for each of 1 candidates, totalling 6 fits
--- GridSearchCV 完成 ---
最佳 Ridge Alpha: 1000
对应的最佳 CV MAE (原始价格): 123,206.70

--- Ridge 模型性能 ('rent' data, MAE in Original Price) ---
Best Alpha:        1000
In-sample MAE:     122,038.54
Out-of-sample MAE (local validation): 123,175.82
6-Fold CV MAE:     123,206.70

--- Ridge 结果表格 ('rent' data, 用于报告) ---
| Metrics   |   In sample |   out of sample |   Cross-validation |
|:----------|------------:|----------------:|-------------------:|
| Ridge     |  122,038.54 |      123,175.82 |         123,206.70 |

--- 7. 正在使用最佳 Alpha 重新训练模型并生成 prediction_rent_ridge.csv 文件... ---
预测结果已保存到 'prediction_rent_ridge.csv'

--- Ridge 模型 ('rent' data) 处理完毕 ---


In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso # <-- 更改为 Lasso
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import warnings

warnings.filterwarnings('ignore')

# 确保 category_encoders 已安装
try:
    from category_encoders import TargetEncoder
except ImportError:
    print("错误: TargetEncoder 未找到。")
    print("请先在您的环境中运行: pip install category_encoders")
    TargetEncoder = None # Define as None if not found

# --- (假设 train_df_final 和 test_df_final 已在内存中) ---

print("--- 1. 准备训练和测试数据 ('rent' data) ---")

# a. 分离训练集的 X 和 y
if 'log_price' in train_df_final.columns:
    X_train_full = train_df_final.drop('log_price', axis=1)
    y_train_full = train_df_final['log_price']
else:
    raise ValueError("'log_price' 列未在最终训练集中找到！")

# b. 分离测试集的 X 和 ID
if 'ID' in test_df_final.columns:
    X_test = test_df_final.drop('ID', axis=1)
    test_ids = test_df_final['ID']
else:
    print("警告: 最终测试集中未找到 ID 列。将生成虚拟 ID。")
    X_test = test_df_final.copy()
    test_ids = pd.Series(range(len(X_test)), name="ID")

# c. 验证列匹配 (保险起见)
if not all(X_train_full.columns == X_test.columns):
    print("警告: 训练集和测试集列不匹配，尝试重新对齐...")
    train_cols_set = set(X_train_full.columns)
    test_cols_set = set(X_test.columns)
    if train_cols_set != test_cols_set:
         raise ValueError("训练集和测试集的特征列不匹配！请检查处理流程。")
    else:
         X_test = X_test[X_train_full.columns]
         print("列已对齐。")

# d. 转换 TargetEncoder 列为 category 类型
target_encode_cols = ['区县', '板块'] # Rent 数据需要 Target Encoding 的列
target_encode_cols = [col for col in target_encode_cols if col in X_train_full.columns]
if target_encode_cols and TargetEncoder is None:
     raise ImportError("TargetEncoder is required but not found.")
if target_encode_cols:
    print(f"--- 正在转换 {target_encode_cols} 为 'category' 类型 ---")
    for col in target_encode_cols:
        X_train_full[col] = X_train_full[col].astype('category')
        X_test[col] = X_test[col].astype('category')

print(f"X_train_full 维度: {X_train_full.shape}")
print(f"X_test 维度: {X_test.shape}")

# --- 2. 创建预处理管道 (与 OLS/Ridge 脚本相同) ---
print("\n--- 2. 正在创建预处理管道... ---")
numeric_cols = [col for col in X_train_full.columns if col not in target_encode_cols]
transformers = []
if target_encode_cols:
    transformers.append(('target_encoder', TargetEncoder(), target_encode_cols))
transformers.append(('standard_scaler', StandardScaler(), numeric_cols))
preprocessor = ColumnTransformer(transformers=transformers, remainder='passthrough')

# --- 3. 定义 Lasso 模型和超参数网格 ---
print("\n--- 3. 正在定义 Lasso 模型和超参数网格... ---")
# (创建 Pipeline，Regressor 更改为 Lasso)
pipeline_lasso = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Lasso(random_state=111, max_iter=5000)) # <-- 使用 Lasso，增加 max_iter
])

# 定义要搜索的 alpha (L1 正则化强度) 值
# 注意: Lasso 的 alpha 通常需要设置得非常小
param_grid_lasso = {
    'regressor__alpha': [0.0001] # <-- 示例网格，您可能需要调整
}

# --- 4. 使用 GridSearchCV 寻找最佳 Alpha ---
print("\n--- 4. 正在使用 GridSearchCV 寻找最佳 Lasso Alpha... ---")

# 定义 MAE 评分器 (与之前相同)
def original_price_mae_scorer(y_log, y_pred_log):
    y_orig = np.expm1(y_log)
    y_pred_orig = np.expm1(y_pred_log)
    y_pred_orig = np.nan_to_num(y_pred_orig, nan=0.0, posinf=np.finfo(np.float64).max, neginf=0.0)
    y_pred_orig = np.clip(y_pred_orig, 0, None)
    return mean_absolute_error(y_orig, y_pred_orig)

neg_mae_scorer = make_scorer(original_price_mae_scorer, greater_is_better=False)

# 使用 6 折交叉验证
kf = KFold(n_splits=6, shuffle=True, random_state=111)

grid_search_lasso = GridSearchCV(
    pipeline_lasso,
    param_grid_lasso,
    cv=kf,
    scoring=neg_mae_scorer,
    n_jobs=-1,
    verbose=1 # 显示进度
)

# 在完整的训练集上运行 GridSearch
grid_search_lasso.fit(X_train_full, y_train_full)

# 获取最佳模型和最佳 alpha
best_model_lasso = grid_search_lasso.best_estimator_
best_alpha_lasso = grid_search_lasso.best_params_['regressor__alpha']

print(f"--- GridSearchCV 完成 ---")
print(f"最佳 Lasso Alpha: {best_alpha_lasso}")
print(f"对应的最佳 CV MAE (原始价格): {-grid_search_lasso.best_score_:,.2f}")

# --- 5. 计算指标 (使用最佳模型) ---
print("\n--- 5. 正在计算 Lasso 指标... ---")

# a. In-sample (样本内) MAE
y_train_orig = np.expm1(y_train_full) # 真实的原始价格
y_pred_train_log = best_model_lasso.predict(X_train_full)
y_pred_train_orig = np.expm1(y_pred_train_log)
mae_in_sample = mean_absolute_error(y_train_orig, y_pred_train_orig)

# b. Out-of-sample (样本外) MAE - 使用临时的 test split
X_train_temp, X_val_temp, y_train_temp, y_val_temp = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=111
)
best_model_lasso.fit(X_train_temp, y_train_temp) # 用最佳 alpha 重新训练 (以正确fit TargetEncoder)
y_pred_val_log = best_model_lasso.predict(X_val_temp)
mae_out_of_sample = mean_absolute_error(np.expm1(y_val_temp), np.expm1(y_pred_val_log))
del X_train_temp, X_val_temp, y_train_temp, y_val_temp # 清理

# c. 6-fold Cross-validation MAE (GridSearch 已经计算过了)
mae_cv = -grid_search_lasso.best_score_

# --- 6. 报告结果 ---
print("\n--- Lasso 模型性能 ('rent' data, MAE in Original Price) ---")
print(f"Best Alpha:        {best_alpha_lasso}")
print(f"In-sample MAE:     {mae_in_sample:,.2f}")
print(f"Out-of-sample MAE (local validation): {mae_out_of_sample:,.2f}")
print(f"6-Fold CV MAE:     {mae_cv:,.2f}")

lasso_results_rent = {
    'Metrics': 'Lasso',
    'In sample': mae_in_sample,
    'out of sample': mae_out_of_sample, # 本地验证集
    'Cross-validation': mae_cv
}

print("\n--- Lasso 结果表格 ('rent' data, 用于报告) ---")
print(pd.DataFrame([lasso_results_rent]).to_markdown(index=False, floatfmt=",.2f"))

# --- 7. 生成预测文件 (prediction_rent_lasso.csv) ---
print("\n--- 7. 正在使用最佳 Alpha 重新训练模型并生成 prediction_rent_lasso.csv 文件... ---")

# a. 使用找到的最佳 Alpha 在 *完整* 训练集上训练最终模型
# (GridSearch 默认会用最佳参数在完整数据上 refit)
final_model_lasso = grid_search_lasso.best_estimator_ 

# b. 预测测试集
y_pred_test_log = final_model_lasso.predict(X_test)

# c. 反向转换并处理异常值
y_pred_test_orig = np.expm1(y_pred_test_log)
fallback_price = np.expm1(y_train_full.median()) # 使用训练集的中位数租金作为 fallback
y_pred_test_orig = np.nan_to_num(y_pred_test_orig, nan=fallback_price, posinf=np.finfo(np.float64).max, neginf=0.0)
y_pred_test_orig = np.clip(y_pred_test_orig, 0, None) # 确保租金非负

# d. 创建提交 DataFrame
submission_df_lasso_rent = pd.DataFrame({'ID': test_ids, 'Price': y_pred_test_orig})

# e. 保存为 CSV 文件
submission_filename_lasso_rent = 'prediction_rent_lasso.csv'
submission_df_lasso_rent.to_csv(submission_filename_lasso_rent, index=False)

print(f"预测结果已保存到 '{submission_filename_lasso_rent}'")
print("\n--- Lasso 模型 ('rent' data) 处理完毕 ---")

--- 1. 准备训练和测试数据 ('rent' data) ---
--- 正在转换 ['区县', '板块'] 为 'category' 类型 ---
X_train_full 维度: (98890, 141)
X_test 维度: (9773, 141)

--- 2. 正在创建预处理管道... ---

--- 3. 正在定义 Lasso 模型和超参数网格... ---

--- 4. 正在使用 GridSearchCV 寻找最佳 Lasso Alpha... ---
Fitting 6 folds for each of 1 candidates, totalling 6 fits
--- GridSearchCV 完成 ---
最佳 Lasso Alpha: 0.0001
对应的最佳 CV MAE (原始价格): 120,143.04

--- 5. 正在计算 Lasso 指标... ---

--- Lasso 模型性能 ('rent' data, MAE in Original Price) ---
Best Alpha:        0.0001
In-sample MAE:     119,219.66
Out-of-sample MAE (local validation): 120,216.47
6-Fold CV MAE:     120,143.04

--- Lasso 结果表格 ('rent' data, 用于报告) ---
| Metrics   |   In sample |   out of sample |   Cross-validation |
|:----------|------------:|----------------:|-------------------:|
| Lasso     |  119,219.66 |      120,216.47 |         120,143.04 |

--- 7. 正在使用最佳 Alpha 重新训练模型并生成 prediction_rent_lasso.csv 文件... ---
预测结果已保存到 'prediction_rent_lasso.csv'

--- Lasso 模型 ('rent' data) 处理完毕 ---
